# Homogenization - main

This software aims at providing a user-friendly tool for homogeneization of a matrix containing fillers. 
Our goal was to develop a tool that could:
- Compute the linear elastic and linear viscoelastic behaviors of microstructures defined by geometry and constitutive phases behaviors,
- Compare models for a given microstructure 
- Offer an inverse method to reach microstructure parameters such as filler volume fraction, phase mechanical properties when knowing the target homogenized behavior
- Provide comprehensive descriptions of the implemented models emphasizing their relevance and llimitation. 
- Provide with an evolutive tool that anyone could increment with other models or microstructures

**This notebooks works independently from classes_v4.py, fast_ellipsoids_functions. It also does not need the compilation of a f2py module.** 
You can use it directly on notebooks.ai, or locally **if you have added gfortran to your Path** (if you can compile a script anywhere with your terminal).

This code contains multiple sections (one per main feature). **When the script is run for the first time on a new kernel, the following sections have to be executed according to this order**:
- I- Classes and modules import
- II- Useful functions

The other sections are independant.

---

# I- Classes and modules import

In [1]:
!pip3 install ipywidgets # Installation of the package managing widgets
import ipywidgets as widgets
from IPython.display import clear_output, display, Markdown
clear_output()
from os import listdir
import pandas as pd
import csv

## Replacement of fortran_tools module with some functions

### If you are using Notebooks.ai

In [2]:
! apt install -y gfortran     
!pip install -U fortran-magic
clear_output()
%load_ext fortranmagic

### If you are working locally with gfortran already installed

In [3]:
!pip install -U fortran-magic
clear_output()
%load_ext fortranmagic
%matplotlib widget

ModuleNotFoundError: No module named 'ipympl'

### Useful fortran functions

In [3]:
%%fortran -v
        subroutine fast_tensor_rotation(A , R, B)
        real, dimension(81),intent(in) :: A
        real, dimension(3,3),intent(in) :: R
        real ::  product,bijkl,term
        real, dimension(81), intent(out) :: B

        do i = 0,2
        do j = 0,2
        do k = 0,2
        do l = 0,2
            B(27*i + 9*j + 3*k + l + 1) = 0
        end do
        end do
        end do
        end do

        do i=0,2
        do j = 0,i
        do k = 0,2
        do l = 0,k    
         do m = 0,2
         do n = 0,2
         do ll = 0,2
         do kk = 0,2
          product = R(i+1,m+1)* R(j+1,n+1)*R(k+1,ll+1)*R(l+1,kk+1)
          bijkl =  B(27*i + 9*j + 3*k + l + 1)
          term = bijkl + A(27*m + 9*n +3*ll + kk +1)*product
          B(27*i + 9*j + 3*k + l + 1) =  term
         end do
         end do
         end do
         end do
         B(27*j + 9*i + 3*k + l + 1) = B(27*i + 9*j + 3*k + l + 1)
         B(27*i + 9*j + 3*l + k + 1) = B(27*i + 9*j + 3*k + l + 1)
         B(27*j + 9*i + 3*l + k + 1) = B(27*i + 9*j + 3*k + l + 1)
        end do
        end do
        end do
        end do
        end subroutine fast_tensor_rotation


Ok. The following fortran objects are ready to use: fast_tensor_rotation


In [4]:
%%fortran -v     
        subroutine inversionT(A,S)
        real, dimension(3,3),intent(in) :: A
        real, dimension(3,3),intent(out) :: S
        real :: det,a1,a2,a3,d1,d2,d3

        
        a1 = A(1,1)*A(2,2)*A(3,3)
        a2 = A(1,2)*A(2,3)*A(3,1)
        a3 = A(1,3)*A(2,1)*A(3,2) 
        d1 =  A(3,2)*A(2,3)*A(1,1)
        d2 =  A(3,3)*A(2,1)*A(1,2)
        d3 =  A(3,1)*A(2,2)*A(1,3)
        det = a1+a2+a3 - d1-d2-d3 

        S(1,1) = (A(2,2)*A(3,3)-A(3,2)*A(2,3))/det
        S(2,1) = -(A(1,2)*A(3,3)-A(3,2)*A(1,3))/det
        S(3,1) = (A(1,2)*A(2,3)-A(2,2)*A(1,3))/det

        S(1,2) = -(A(2,1)*A(3,3)-A(3,1)*A(2,3))/det
        S(2,2) = (A(1,1)*A(3,3)-A(3,1)*A(1,3))/det
        S(3,2) = -(A(1,1)*A(2,3)-A(2,1)*A(1,3))/det

        S(1,3) = (A(2,1)*A(3,2)-A(3,1)*A(2,2))/det
        S(2,3) = -(A(1,1)*A(3,2)-A(3,1)*A(1,2))/det
        S(3,3) = (A(1,1)*A(2,2)-A(2,1)*A(1,2))/det


        end subroutine inversionT


Ok. The following fortran objects are ready to use: inversiont


### Manual compilation of fast_ellipsoids_functions without fortran_tools modulus

In [5]:
'''
Useful functions for ellipsoid calculation
Uses fortran_tools which must have been compiled with f2py previously
'''
import numpy as np
from numpy import pi,cos,sin,arccos,arcsin
from numpy.random import random_sample
from numpy.linalg import inv
from numpy import dot
from scipy.spatial.transform import Rotation as Rot



def Comp3333_to_66 (G) : 
    "Returns from a behaviour tensor G 3x3x3 to a behaviour matrix F 6x6"
    F=np.zeros((6,6))
    for i in range(3):
        for j in range(3):
            F[i,j] = G[i,i,j,j]
            
        F[i,5]=(G[i,i,0,1]+G[i,i,1,0])/2.
        F[i,3]=(G[i,i,1,2]+G[i,i,2,1])/2. 
        F[i,4]=(G[i,i,2,0]+G[i,i,0,2])/2. 
        F[3,i]=(G[1,2,i,i]+G[2,1,i,i])/2. 
        F[4,i]=(G[0,2,i,i]+G[2,0,i,i])/2.
        F[5,i]=(G[0,1,i,i]+G[1,0,i,i])/2.

    F[4,4]=(G[0,2,0,2]+G[2,0,0,2]+G[0,2,2,0]+G[2,0,2,0])/4. 
    F[3,3]=(G[1,2,1,2]+G[2,1,1,2]+G[1,2,2,1]+G[2,1,2,1])/4.  
    F[5,5]=(G[0,1,0,1]+G[1,0,0,1]+G[0,1,1,0]+G[1,0,1,0])/4.  
    F[4,3]=(G[0,2,1,2]+G[2,0,1,2]+G[0,2,2,1]+G[2,0,2,1])/4.  
    F[4,5]=(G[0,2,1,0]+G[2,0,1,0]+G[0,2,0,1]+G[2,0,0,1])/4.  
    F[3,4]=(G[1,2,0,2]+G[2,1,0,2]+G[1,2,2,0]+G[2,1,2,0])/4.  
    F[5,4]=(G[0,1,0,2]+G[1,0,0,2]+G[0,1,2,0]+G[1,0,2,0])/4.  
    F[3,5]=(G[1,2,1,0]+G[2,1,1,0]+G[1,2,0,1]+G[2,1,0,1])/4.   
    F[5,3]=(G[0,1,1,2]+G[1,0,1,2]+G[0,1,2,1]+G[1,0,2,1])/4. 
    
    return F

def Comp66_to_3333(F) : 
    ' Returns a matrix F 6x6 from a behaviour tensor G 3x3x3x3'
    G = np.zeros((3,3,3,3))
    for i in range(3) :
        for j in range(3) :
            G[i,i,j,j]=F[i,j]
       
        G[i,i,0,1]=F[i,5]
        G[i,i,1,2]=F[i,3]
        G[i,i,2,0]=F[i,4]
        G[0,2,i,i]=F[4,i]
        G[1,2,i,i]=F[3,i]
        G[0,1,i,i]=F[5,i]
        G[i,i,1,0]=F[i,5]
        G[i,i,2,1]=F[i,3]
        G[i,i,0,2]=F[i,4]
        G[2,0,i,i]=F[4,i]
        G[2,1,i,i]=F[3,i]
        G[1,0,i,i]=F[5,i]
        
    G[0,1,0,1]=F[5,5]
    G[0,1,0,2]=F[5,4]
    G[0,1,1,0]=F[5,5]
    G[0,1,1,2]=F[5,3] 
    G[0,1,2,0]=F[5,4]
    G[0,1,2,1]=F[5,3]

    G[0,2,0,1]=F[4,5]
    G[0,2,0,2]=F[4,4]
    G[0,2,1,0]=F[4,5]
    G[0,2,1,2]=F[4,3] 
    G[0,2,2,0]=F[4,4]
    G[0,2,2,1]=F[4,3]

    G[1,0,0,1]=F[5,5]
    G[1,0,0,2]=F[5,4]
    G[1,0,1,0]=F[5,5]
    G[1,0,1,2]=F[5,3] 
    G[1,0,2,0]=F[5,4]
    G[1,0,2,1]=F[5,3]

    G[1,2,0,2]=F[3,4]
    G[1,2,1,0]=F[3,5]
    G[1,2,1,2]=F[3,3] 
    G[1,2,2,0]=F[3,4]
    G[1,2,2,1]=F[3,3]

    G[2,0,0,1]=F[4,5]
    G[2,0,0,2]=F[4,4]
    G[2,0,1,0]=F[4,5]
    G[2,0,1,2]=F[4,3] 
    G[2,0,2,0]=F[4,4]
    G[2,0,2,1]=F[4,3]

    G[2,1,0,1]=F[3,5]
    G[2,1,0,2]=F[3,4]
    G[2,1,1,0]=F[3,5]
    G[2,1,1,2]=F[3,3] 
    G[2,1,2,0]=F[3,4]
    G[2,1,2,1]=F[3,3]
 
    return G 

def Rotation_matrices(n) : 
    'Create a matrix nx3x3 composed of n rotation matrix with  three euler angles taken such as orientation are evenly distributed '
    Q = np.zeros((n,3,3))
    for i in range(n) : 
        theta,phi,psi = Rot.random().as_euler('zxy', degrees=False)
        Q[i,0,0]=cos(psi)*cos(theta)-cos(phi)*sin(theta)*sin(psi)
        Q[i,0,1]=sin(theta)*cos(psi)+cos(phi)*sin(psi)*cos(theta)
        Q[i,0,2]=sin(phi)*sin(psi)
        Q[i,1,0]=-sin(psi)*cos(theta)-sin(theta)*cos(phi)*cos(psi)
        Q[i,1,1]=cos(psi)*cos(phi)*cos(theta)-sin(theta)*sin(psi)
        Q[i,1,2]=cos(psi)*sin(phi)
        Q[i,2,0]=sin(phi)*sin(theta)
        Q[i,2,1]=-sin(phi)*cos(theta)
        Q[i,2,2]=cos(phi)
    
        for j in range(3) : 
            for k in range(3):
                if (abs(Q[i,j,k]) < 10**-6 ) :
                    Q[i,j,k] = 0
            
    return Q

def Rotation_operator(n_renforts) : 
    'Create a n*3**8 matrix which contains the product necessary to execute Rotation_tensor'
    B = np.zeros((n_renforts,3,3,3,3,3,3,3,3))
    R = np.zeros((n_renforts,3,3))
    for z in range(n_renforts) :
        theta,phi,psi = Rot.random().as_euler('zxy', degrees=False)
        R[z,0,0]=cos(psi)*cos(theta)-cos(phi)*sin(theta)*sin(psi)
        R[z,0,1]=sin(theta)*cos(psi)+cos(phi)*sin(psi)*cos(theta)
        R[z,0,2]=sin(phi)*sin(psi)
        R[z,1,0]=-sin(psi)*cos(theta)-sin(theta)*cos(phi)*cos(psi)
        R[z,1,1]=cos(psi)*cos(phi)*cos(theta)-sin(theta)*sin(psi)
        R[z,1,2]=cos(psi)*sin(phi)
        R[z,2,0]=sin(phi)*sin(theta)
        R[z,2,1]=-sin(phi)*cos(theta)
        R[z,2,2]=cos(phi)
        for  i in range(3) : 
            for  j in range(i+1):
                for  k in range(3):
                    for  l in range(k+1):
                        for  m in range(3):
                            for  n in range(3):
                                for  ll in range(3):
                                    for  kk in range(3):
                                        B[z,i,j,k,l,m,n,ll,kk] = R[z,i,m]*R[z,j,n]*R[z,k,ll]*R[z,l,kk]
    return B



def Rotation_tensor(S,Operator,z,B) : 
    ' Returns the rotation of the tensor S through the 3 Euler angles taken randomly '
    
    for  i in range(3) : 
        for  j in range(3):
            for  k in range(3):
                for  l in range(3):
                    B[i,j,k,l] = 0
                    
    for  i in range(3) : 
        for  j in range(i+1):
            for  k in range(3):
                for  l in range(k+1):
                    for  m in range(3):
                        for  n in range(3):
                            for  ll in range(3):
                                for  kk in range(3):
                                    B[i,j,k,l] += Operator[z,i,j,k,l,m,n,ll,kk]*S[m,n,ll,kk]                                    
  
                    B[i,j,l,k] = B[i,j,k,l]
                    B[j,i,k,l] = B[i,j,k,l]
                    B[j,i,l,k] = B[i,j,k,l]
    return B


def Isotropic_Compliance_Matrix(E,nu) :
    'Returns the compliance matrix of an isotropic material'
    S = np.zeros((6,6))
    S[0,0]=1./E
    S[1,1]=1./E
    S[2,2]=1./E

    S[3,3]=2.*(1+nu)/E
    S[4,4]=2.*(1+nu)/E
    S[5,5]=2.*(1+nu)/E

    S[0,1]=-nu/E
    S[0,2]=-nu/E
    S[1,2]=-nu/E
    S[1,0]=-nu/E
    S[2,1]=-nu/E
    S[2,0]=-nu/E
    
    return S
    

def isotropic_young(S) : 
    return 1/3 * (1/S[0,0]+1/S[1,1]+1/S[2,2])

def isotropic_nu(S) : 
    E = isotropic_young(S)
    return (min(- 1/6 * E * (S[0,1] + S[0,2] + S[1,2] + S[1,0] + S[2,0] + S[2,1]),0.499999999))

def isotropic_young_C(C) : 
    nu = isotropic_nu_C(C)
    return 2 * (1+nu) * 1/3 *(C[3,3]+C[4,4]+C[5,5])

def isotropic_nu_C(C) : 
    x = 2 * (C[0,0]+C[1,1]+C[2,2]) / (C[0,1]+C[0,2]+C[1,2]+C[1,0]+C[2,0]+C[2,1])
    return (min(1/(1+x),0.499999999))

    
def Young_anisotrope(S) : 
    return 1/S[0,0],1/S[1,1],1/S[2,2]



def clear_matrix3 (C,k) : 
    n = C.shape[0]
    for i in range(n) : 
        for j in range(n) :
            if C[i,j,k]<10**-8 : 
                C[i,j,k] = 0
                
def clear_matrix2 (C) : 
    n = C.shape[0]
    for i in range(n) : 
        for j in range(n) :
            if C[i,j]<10**-5 : 
                C[i,j] = 0
                

def Eshelby_tensor(Axis,Cm,Sm) : 
    
    Cm3 = Comp66_to_3333(Cm)
    a0,a1,a2 = Axis
    IJV = np.array([[0,0],[1,1],[2,2],[1,2],[0,2],[0,1]])
    Nit = 40
    Ntop = Nit
    Mtop = Nit
    dphi = pi/(Ntop-1)
    dtheta = pi/(Ntop-1)
    A = np.zeros((6,6))
    B = np.zeros((6,6,Mtop))
    G = np.zeros((6,6,Ntop))
    E = np.zeros((6,6))
    
    # Integration de la fonction de green sur la demi ellipsoïde
    for m in range(Mtop) : 
        phi = m*dphi
        for n in range(Ntop) : 
            theta = n*dtheta
            X = np.array([sin(theta)*cos(phi)/a0 , sin(theta)*sin(phi)/a1 , cos(theta)/a2])
            CXX = np.zeros((3,3))
            for i in range(3) :
                for j in range(3) :
                    for k in range(3) : 
                        for l in range(3) :
                            CXX[i,k] += Cm3[i,j,k,l]*X[j]*X[l]
            CXX = inv(CXX)
            for i in range(6) :
                for j in range(6) :                     
                    I1 = IJV[i,0]
                    J1 = IJV[j,0]
                    I2 = IJV[i,1]
                    J2 = IJV[j,1]
                    G[i,j,n] = 0.5 * sin(theta) * (CXX[I1,J1]*X[I2]*X[J2] + CXX[I2,J1]*X[I1]*X[J2] + CXX[I1,J2]*X[I2]*X[J1] + CXX[I2,J2]*X[I1]*X[J1])
        
        
        B[:,:,m] = 0.5 * dtheta * (G[:,:,0]+G[:,:,Ntop-1])
        for i in range(1,Ntop-1) : 
            B[:,:,m] +=  dtheta * G[:,:,i]

    A = 0.5*(B[:,:,0]+B[:,:,Ntop-1])* dphi/(4*pi)
    for i in range(1,Ntop-1) : 
         A += B[:,:,i]* dphi/(4*pi)  
    
    for i in range(6) : 
        for j in range(6) : 
            E[i,j]=A[i,0]*Cm[0,j]+A[i,1]*Cm[1,j]+A[i,2]*Cm[2,j] + 4* (A[i,3]*Cm[3,j]+A[i,4]*Cm[4,j]+A[i,5]*Cm[5,j]) 
    
    return E

def Fast_Eshelby_tensor(Axis,Cm,Sm) : 
    
    Cm3 = Comp66_to_3333(Cm)
    a0,a1,a2 = Axis
    IJV = np.array([[0,0],[1,1],[2,2],[1,2],[0,2],[0,1]])
    Nit = 40
    Ntop = Nit
    Mtop = Nit
    dphi = pi/(Ntop-1)
    dtheta = pi/(Ntop-1)
    A = np.zeros((6,6))
    B = np.zeros((6,6,Mtop))
    G = np.zeros((6,6,Ntop))
    E = np.zeros((6,6))
    
    # Integration de la fonction de green sur la demi ellipsoïde
    for m in range(Mtop) : 
        phi = m*dphi
        for n in range(Ntop) : 
            theta = n*dtheta
            X = np.array([sin(theta)*cos(phi)/a0 , sin(theta)*sin(phi)/a1 , cos(theta)/a2])
            TCXX = np.zeros((3,3))
            for i in range(3) :
                for j in range(3) :
                    for k in range(3) : 
                        for l in range(3) :
                            TCXX[i,k] += Cm3[k,j,i,l]*X[j]*X[l] ## Calcul de la transposée de CXX
             
            CXX = inversiont(TCXX)  ## Calcul de l'inverse de CXX par Fortran
            
            for i in range(6) :
                for j in range(6) :                     
                    I1 = IJV[i,0]
                    J1 = IJV[j,0]
                    I2 = IJV[i,1]
                    J2 = IJV[j,1]
                    G[i,j,n] = 0.5 * sin(theta) * (CXX[I1,J1]*X[I2]*X[J2] + CXX[I2,J1]*X[I1]*X[J2] + CXX[I1,J2]*X[I2]*X[J1] + CXX[I2,J2]*X[I1]*X[J1])
        
        
        B[:,:,m] = 0.5 * dtheta * (G[:,:,0]+G[:,:,Ntop-1])
        for i in range(1,Ntop-1) : 
            B[:,:,m] +=  dtheta * G[:,:,i]

    A = 0.5*(B[:,:,0]+B[:,:,Ntop-1])* dphi/(4*pi)
    for i in range(1,Ntop-1) : 
         A += B[:,:,i]* dphi/(4*pi)  
    
    for i in range(6) : 
        for j in range(6) : 
            E[i,j]=A[i,0]*Cm[0,j]+A[i,1]*Cm[1,j]+A[i,2]*Cm[2,j] + 4* (A[i,3]*Cm[3,j]+A[i,4]*Cm[4,j]+A[i,5]*Cm[5,j]) 
    
    return E 

def Matrix_to_vecteur(A) : 
    L = np.zeros(81)
    for i in range(3) : 
        for j in range(3) : 
            for k in range(3):
                for l in range(3) : 
                    L[27*i + 9*j + 3*k + l] = A[i,j,k,l]
    return L

def Vecteur_to_matrix(L) : 
    A = np.zeros((3,3,3,3))
    for i in range(3) : 
        for j in range(3) : 
            for k in range(3):
                for l in range(3) : 
                    A[i,j,k,l] = L[27*i + 9*j + 3*k + l]
    return A

### Manual compilation of classes_v5 without fortran_tools modulus

In [6]:
# -*- coding: utf-8 -*-
"""
Homogeneisation - classes.py

Definition of classes used in Fast_homogenisation_main. Includes a call to a module fortran previously compiled.

Authors : Karim AÏT AMMAR, Enguerrand LUCAS

11/06/2020
"""

#%% Import packages
import numpy as np
from scipy import *
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from math import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#%% Microstructure classes
class Inclusion:
    """
    Contains info of the inclusion (type, behavior, shape).
    """
    
    def __init__(self, type_inclusion, behavior, aspect_ratio=[1.,1.], name=None, frequency=[], abscissa="frequency"):
        """
    
        type_inclusion: (int), 0 for one spherical isotropic inclusion, 1 for isotropic or anisotropic ellipsoids and spheres
        aspect_ratio: (tuple), tuple of two floats representing the length ratio of axis 2 and 3 of the ellipsoid to the length of the axis 1 
        behavior: (dict), contains values of matrix behavior : E,K,G and nu in the isotropic case, compliance and stiffness matrix in the anisotropic
        frequency: (list), list of frequencies/temperatures associated with visco-elastic parameters
        abscissa: (str),  "frequency" or "temperature", it indicates the physical nature of the values in the frequency list
        inc_and_int: ([InclusionAndInterphase,int]), None by default, linked to the InclusionAndInterphase class instance to which the inclusion belongs if it exists, and an integer, 0 for the inclusion, 1 for the interphase
        """
        self.type_inclusion = type_inclusion
        self.aspect_ratio = aspect_ratio
        self.behavior = complete_behavior(behavior)
        self.name = name
        self.frequency = frequency
        self.abscissa = abscissa
        self.inc_and_int = None
    
    def type_to_str(self):
        """
        Transforms an integer "type_inclusion" into the corresponding string (example: 0 --> "spheres") 
        """
        type_inclusion = self.type_inclusion
        try:
            result = dict_types[type_inclusion]
        except KeyError:
            # Unspecified type in dictionnary
            result = None
        return result
    
    def __str__(self):
        """
        Presentation of the instance.
        """
        str_type_inclusion = self.type_to_str()
        string = "{}, {}".format(self.name, str_type_inclusion)
        if self.type_inclusion != 0:
            string += " (ratios={})".format(self.aspect_ratio)
        for parameter, value in self.behavior.items():
            if type(value) not in [list, np.ndarray]:
                string += ", {}: {:.2f}".format(parameter, value)
            else:
                string += ", {}: list".format(parameter)
        return string

    def __repr__(self):
        return str(self)

    def change_parameter(self, parameter, new_value):
        """
        Changes the value of the parameter if it exists. Updates the behavior with the function "complete_behavior".
        """
        try:
            self.behavior[parameter] = new_value
            self.behavior = complete_behavior(self.behavior)
        except:
            None

    def graph_parameter(self):
        """
       Plots the graph of the evolution of the visco-elastic parameters, if they exist
        """
        if self.frequency == []:
            None # L'inclusion ne contient pas de paramètres visco-élastiques
        else:
            plt.figure()
            # for parameter, values in self.behavior.items():
            for parameter in ['K', 'G']:
                values = self.behavior[parameter]
                # Le paramètre est visco-élastique
                if self.abscissa == "temperature":
                    plt.semilogy(self.frequency, values.real, '.', label=parameter+"'")
                    plt.semilogy(self.frequency, values.imag, '.', label=parameter+"''")
                elif self.abscissa == "frequency":
                    plt.loglog(self.frequency, values.real, '.', label=parameter+"'")
                    plt.loglog(self.frequency, values.imag, '.', label=parameter+"''")
                plt.legend()
                plt.xlabel(self.abscissa)
                plt.ylabel("Parameter value")
                plt.title("Inclusion visco-elastic behavior")
                plt.xlim(min(self.frequency), max(self.frequency))
            plt.show()

class InclusionAndInterphase:
    """
    Instance representing an inclusion and the associated interphase. Currently only used in the Self-consistent 4-phases model
    """            
    
    def __init__(self, inclusion, interphase, name=None):
        """
        Inclusion: Inclusion Class Instance
        interphase: Inclusion class instance, represents the interphase associated with inclusion.
        The interphase and the inclusion must be of the same type (spheres or ellipsoids with the same aspect ratios).
        """
        assert inclusion.aspect_ratio==interphase.aspect_ratio
        self.inclusion = inclusion
        self.interphase = interphase
        self.name = name
        self.aspect_ratio = inclusion.aspect_ratio
        # Modification de l'attribut inc_and_int des inclusion et interphase
        inclusion.inc_and_int = [self, 0]
        interphase.inc_and_int = [self, 1]
        
    def __str__(self):
        string = "Inclusion + Interphase\n"
        string += "Inclusion: {}\n".format(str(self.inclusion))
        string += "Interphase: " + str(self.interphase)
        return string
    
    def __repr__(self):
        return str(self)

class Microstructure:
    """
    Contains information on the microstructure (matrix behaviour and inclusions)
    Contains a function that returns the Hashin-Shtrickman bounds for a microstructure with one spherical isotropic inclusion
    """
    
    def __init__(self, behavior, dict_inclusions=dict(), frequency=[], abscissa="frequency"):
        """
        list_inclusions: (dictionnary such as {inclusion: f_i} with inclusion as an instance of class Inclusion and f_i as the volume fraction of this type of inclusion. inclusion can also be an instance of class InclusionAndInterphase, in this case, f_i is a tuple of floaters
        behavior: (dict), contains the values of the parameters of the behavior matrix (example : {K:10, G:10 , E:22.5 ,nu:0.12})
        frequency: list of frequencies associated with viscoelastic parameters
        """
        self.dict_inclusions = dict_inclusions
        self.behavior = complete_behavior(behavior)
        # Calcul de la fraction volumique de matrice f_m
        self.f_matrix = self.compute_fm()
        self.frequency = frequency
        self.abscissa = abscissa
        
    def __str__(self):
        string = "Microstructure\nf_m = {:.2f}, matrix".format(self.f_matrix, self.behavior)
        for parameter, value in self.behavior.items():
            if type(value) not in [list, np.ndarray]:
                string += ", {}: {:.2f}".format(parameter, value)
            else:
                string += ", {}: list".format(parameter)
        dict_inclusions = self.dict_inclusions
        # Display of all inclusion in microstructure
        for inclusion in dict_inclusions.keys():
            fi = dict_inclusions[inclusion]
            string += "\nf_i = {}, ".format(fi) + str(inclusion)
        return string

    def compute_fm(self):
        """
        1/ Checks if the given list of inclusions is consistent (i.e. the sum of  volume fractions inclusions is less  than 1). Else, generates an error.
        2/ If no error is generated, calculates the matrix volume fraction.
        """
        total_fi = 0 # Total of volumic fractions of inclusions
        dict_inclusions = self.dict_inclusions
        for inclusion in dict_inclusions.keys():
            fi = dict_inclusions[inclusion]
            # Case  inclusions + interphase
            if type(fi)==list:
                total_fi += fi[0] + fi[1]
            # Case simple inclusions
            else:
                total_fi += fi
        if total_fi > 1:
            raise NameError("The total volume fractions of the inclusions exceed 1")
        else :
            f_m = 1 - total_fi
            return f_m

    def change_fi(self, inclusion, new_f):
        """
        Updates the volume fraction of the inclusion or adds it to the dictionary if it was not present.
        Updates the volume fraction of the matrix.
        """
        self.dict_inclusions[inclusion] = new_f
        self.f_matrix = self.compute_fm()
    
    def change_parameter(self, parameter, new_value):
        """
        Change the value of the parameter if it exists. Updates the behavior with the function "complete_behavior".
        """
        try:
            self.behavior[parameter] = new_value
            self.behavior = complete_behavior(self.behavior)
        except:
            None

    def graph_parameter(self):
        """
        Plots the graph of the evolution of the visco-elastic parameters if they exist.
        """
        if self.frequency == []:
            None # Inclusion is not visco-elastic
        else:
            plt.figure()
            # for parameter, values in self.behavior.items():
            for parameter in ['K', 'G']:
                values = self.behavior[parameter]
                # Le paramètre est visco-élastique
                if self.abscissa == "temperature":
                    plt.semilogy(self.frequency, values.real, '.', label=parameter+"'")
                    plt.semilogy(self.frequency, values.imag, '.', label=parameter+"''")
                elif self.abscissa == "frequency":
                    plt.loglog(self.frequency, values.real, '.', label=parameter+"'")
                    plt.loglog(self.frequency, values.imag, '.', label=parameter+"''")
                plt.legend()
                plt.xlabel(self.abscissa)
                plt.ylabel("Parameter value")
                plt.title("Matrix visco-elastic behavior")
                plt.xlim(min(self.frequency), max(self.frequency))
            plt.show()

    def draw(self):
        """
         Method for drawing the microstructure.
        """
        inclusions = list(self.dict_inclusions.keys())
        n_fig = len(inclusions)
        if n_fig==0:
            # Microstructure sans inclusion
            return None
        fig = plt.figure(figsize=(n_fig*3 ,3))
        for index, instance in enumerate(inclusions):
            fi = self.dict_inclusions[instance]
            if type(instance)==Inclusion:
                inclusion = instance
                f_inc = fi
                interphase = None
            else:
                inclusion = instance.inclusion
                interphase = instance.interphase
                f_inc = fi[0]
                f_int = fi[1]
                
            ### Draw inclusion
            ax = fig.add_subplot(1, n_fig, index+1, projection='3d')
            # compute radius for a 10X10X10-sized VER
            c1, c2 = inclusion.aspect_ratio
            a = (1000*f_inc/(4/3*pi*c1*c2))**(1/3)
            b = c1*a
            c = c2*a

            # Radii:
            rx, ry, rz = np.array([a, b, c])

            # Set of all spherical angles:
            u = np.linspace(0, 2 * np.pi, 100)
            v = np.linspace(0, np.pi, 100)

            # Cartesian coordinates that correspond to the spherical angles:
            # (this is the equation of an ellipsoid):
            x = rx * np.outer(np.cos(u), np.sin(v))
            y = ry * np.outer(np.sin(u), np.sin(v))
            z = rz * np.outer(np.ones_like(u), np.cos(v))

            # Plot:
            ax.plot_surface(x, y, z,  rstride=4, cstride=4, color='b')
            
            ### Draw interphase
            if interphase!=None:
                a = (1000*(f_inc+f_int)/(4/3*pi*c1*c2))**(1/3)
                b = c1*a
                c = c2*a
    
                # Radii:
                rx, ry, rz = np.array([a, b, c])
    
                # Set of all spherical angles:
                u = np.linspace(0, np.pi, 100)
                v = np.linspace(0, np.pi, 100)
    
                # Cartesian coordinates that correspond to the spherical angles:
                # (this is the equation of an ellipsoid):
                x = rx * np.outer(np.cos(u), np.sin(v))
                y = ry * np.outer(np.sin(u), np.sin(v))
                z = rz * np.outer(np.ones_like(u), np.cos(v))
    
                # Plot:
                ax.plot_surface(x, y, z,  rstride=4, cstride=4, color='r')
            
            ### Draw edges of VER
            # Adjustment of the axes, so that they all have the same span:
            max_radius = 5
            for axis in 'xyz':
                getattr(ax, 'set_{}lim'.format(axis))((-max_radius, max_radius))

            # Cube 
            points = 5*np.array([[-1, -1, -1],
                                  [1, -1, -1 ],
                                  [1, 1, -1],
                                  [-1, 1, -1],
                                  [-1, -1, 1],
                                  [1, -1, 1 ],
                                  [1, 1, 1],
                                  [-1, 1, 1]])

            r = [-5,5]
            X, Y = np.meshgrid(r, r)
            one = 5*np.ones(4).reshape(2, 2)
            ax.plot_wireframe(X,Y,one, alpha=0.5)
            ax.plot_wireframe(X,Y,-one, alpha=0.5)
            ax.plot_wireframe(X,-one,Y, alpha=0.5)
            ax.plot_wireframe(X,one,Y, alpha=0.5)
            ax.plot_wireframe(one,X,Y, alpha=0.5)
            ax.plot_wireframe(-one,X,Y, alpha=0.5)
            ax.scatter3D(points[:, 0], points[:, 1], points[:, 2])
        plt.show()
            
     ## Compute HASHIN-SHTRICKMAN bounds ##########  
    
    def check_Hashin_hypothesis(self) :
        '''
        Return True if there is only one isotropic spherical inclusion
        Else, return False
        '''
        if len(self.dict_inclusions.keys()) != 1 :
            return False
        for inclusion in self.dict_inclusions : 
            if inclusion.type_inclusion != 0 : 
                return False
        return True
            
    
    def khs(k1, g1, c1, k2, g2, c2):
        numerator = c2*(k2-k1)
        denominator = 1+3*c1*(k2-k1)/(4*g1+3*k1)
        return k1+numerator/denominator
    
    def ghs(k1, g1, c1, k2, g2, c2):
        numerator = c2*(g2-g1)
        denominator = 1+6*c1*(g2-g1)*(k1+2*g1)/((3*k1+4*g1)*5*g1)
        return g1+numerator/denominator
        
    def Hashin_bounds(self):
        """
        Gives the Hashin-Shtrikman bounds for single isotropic spherical inclusion
        """
        fm = self.f_matrix
        f = 1-fm
        km,gm = self.behavior["K"], self.behavior["G"]
        if len(list(self.dict_inclusions.keys()))>1:
            return None
        for inclusion in self.dict_inclusions.keys():
            try:
                kf,gf=inclusion.behavior["K"],inclusion.behavior["G"]
            except:
                return None
        
        ksup=max(Microstructure.khs(km,gm,fm,kf,gf,f),Microstructure.khs(kf,gf,f,km,gm,fm))
        kinf=min(Microstructure.khs(km,gm,fm,kf,gf,f),Microstructure.khs(kf,gf,f,km,gm,fm))
        gsup=max(Microstructure.ghs(km,gm,fm,kf,gf,f),Microstructure.ghs(kf,gf,f,km,gm,fm))
        ginf=min(Microstructure.ghs(km,gm,fm,kf,gf,f),Microstructure.ghs(kf,gf,f,km,gm,fm))
            
        
        return { 'Ginf': ginf, 'Gsup': gsup, 'Kinf': kinf, 'Ksup': ksup }
    
    

        

#%% Models classes
class Model:
    """
    Generic parent class of all model classes. 
    Contains the method for verifying the model's assumptions about a microstructure, as well as the method called when calculating the homogenized behavior.
    """
    
    def __str__(self):
        """
         Textual description of the model.
        """
        return self.name + " model"
    
    def __repr__(self):
        """
         Textual description of the model.
        """
        return str(self)
    
    def check_hypothesis(self, microstructure):
        """
         Checks if the microstructure verifies the hypothesis of the model, returns a boolean.
        """
        # Recovery of microstructure inclusions
        behavior_condition = []
        if 'isotropic' in self.behavior_condition:
            behavior_condition.append(set(['K', 'G', 'E', 'nu']))
        if 'anisotropic' in self.behavior_condition:
            behavior_condition.append(set(['C', 'S']))
        # Récupération des inclusions de la microstructure
        dict_inclusions = microstructure.dict_inclusions
        instances = list(dict_inclusions.keys())
        n_instances = len(instances)
        # Initialisation of result
        result = True
        # Checking the number of inclusions
        if n_instances > self.n_inclusions:
             result = False
        # Checking the presence or absence of an interphase
        for instance in instances:
            if (type(instance)==InclusionAndInterphase)!=self.interphase:
                result = False
        # Creation of a list of inclusions without interphase
        inclusions = []
        for instance in instances:
            if type(instance)==InclusionAndInterphase:
                inclusions += [instance.inclusion, instance.interphase]
            else:
                inclusions.append(instance)
        # Checking the type of inclusion
        for inclusion in inclusions:
            if inclusion.type_inclusion > self.type_inclusion:
                result = False
        # Checking the behavior of inclusions and matrix
        for element in inclusions + [microstructure]:
            behav = False
            for behavior_condition0 in behavior_condition:
                if set(element.behavior.keys()).issubset(behavior_condition0):
                    behav = True
            if not behav:    
                result = False
        # Returns result
        return result
    
    def compute_h_behavior(self, microstructure):
        """
        Computes the homogenized behavior of the microstructure with the chosen model.
        Verifies that the model fits the input microstructure.
        If the input elements are not viscoelastic (i.e. if the frequency list of the microstructure is empty), returns a dictionary of real parameters in the form {"parameter": value(float)}. In the isotropic case, also calculates the missing parameters (mu and E, or K and G).
        Otherwise, performs a loop on the frequency values, then returns a complete behaviour dictionary (with missing parameter values) of the form {"parameter": values(complex)]}.
        """
        # Verification of the conditions of application
        compatible = self.check_hypothesis(microstructure)
        if not compatible:
            raise NameError("The microstructure does not match the model hypothesis")
        frequency = microstructure.frequency
        # Creation of the dictionary containing only Inclusions
        inclusions = {}
        for instance, f in microstructure.dict_inclusions.items():
            if type(instance)==InclusionAndInterphase:          ## if the inclusion is an InclusionandInterphase, divide the inclusion into 2 objects
                inclusions[instance.inclusion] = f[0]
                inclusions[instance.interphase] = f[1]
            else:
                inclusions[instance] = f
        # Elastic case
        if not list(frequency):
            Cm = microstructure.behavior
            # Retrieving inclusion behavior in the format : [(inclusion.behavior, f, aspect_ratio)]
            inclusion_behaviors = [(inclusion.behavior, f, inclusion.aspect_ratio) for (inclusion,f) in inclusions.items()]
            ### Calculation of homogenized behaviour
            # Single isotropic sphere inclusion
            if len(inclusion_behaviors)==1 and inclusion_behaviors[0][2]==[1,1] and 'K' in inclusion_behaviors[0][0].keys() and 'K' in Cm.keys():
                h_behavior0 = self.compute_behavior(Cm, inclusion_behaviors)
                h_behavior = {parameter: value.real for (parameter,value) in h_behavior0.items()} # Conversion of possibly complex values into real values
                h_behavior = complete_behavior(h_behavior)
            else:
                # Isotropic behaviors to matrices - inclusions
                inclusion_behaviors1 = []
                for inclusion in inclusion_behaviors:
                    behavior = inclusion[0]
                    if 'C' in behavior:
                        inclusion_behaviors1.append(inclusion)
                    else:
                        E, nu = behavior['E'], behavior['nu']
                        S = Isotropic_Compliance_Matrix(E, nu)
                        C = inv(S)
                        f, ratio = inclusion[1], inclusion[2]
                        inclusion_behaviors1.append(({'C': C, 'S': S}, f, ratio))
                # Isotropic behaviors to matrices - matrix
                if 'K' in Cm.keys():
                    E, nu = Cm['E'], Cm['nu']
                    S = Isotropic_Compliance_Matrix(E, nu)
                    C = inv(S)
                    Cm = {'C': C, 'S': S}
                h_behavior = self.compute_behavior_ellipsoids(Cm, inclusion_behaviors1)
        # Visco-elastic case
        else:
            # Initialisation of result
            h_behavior = {}
            # Calculation of the behavior as a function of frequency
            for i in range(len(frequency)):
                # Recovery of matrix behavior at frequency i
                Cm = {parameter: values[i] for (parameter,values) in microstructure.behavior.items()}
                # Retrieving inclusion behavior at frequency i
                inclusion_behaviors = [] # Initialisation
                for inclusion, f in inclusions.items():
                    inclusion_behavior = {parameter: values[i] for (parameter, values) in inclusion.behavior.items()}
                    inclusion_behaviors.append((inclusion_behavior, f, inclusion.aspect_ratio))
                # Calculation of the homogenized behavior at frequency i
                h_behavior_i = self.compute_behavior(Cm, inclusion_behaviors)
                h_behavior_i = complete_behavior(h_behavior_i)
                # Adds it to the list of behaviors
                for parameter, value in h_behavior_i.items():
                    try:
                        h_behavior[parameter].append(value)
                    except KeyError:
                        # Creation of the input associated with the parameter
                        h_behavior[parameter] = [value]
        # Return of the result
        return h_behavior
    
    
class Voigt_Bound(Model) : 
    
    def __init__(self):
        """
        Definition of model hypotheses.
        """
        self.type_inclusion = 1 # 1 = Model accepts ellipsoidal and spherical inclusion
        self.behavior_condition = ['anisotropic', 'isotropic']  # The model can be applied to microstructures whose inclusions and matrix are isotropic and anisotropic
        self.n_inclusions = 10 # Max number of different types of inclusions
        self.interphase = False # True if the model works on inclusions with interphase
        self.name = "Voigt Bound"
        
    def compute_behavior(self, Cm, inclusion_behaviors):
        
        Cf, f, ratio = inclusion_behaviors[0]
        fm = 1-f
        
        Km,Gm = Cm["K"], Cm["G"]
        Kf,Gf=Cf["K"], Cf["G"]

        K_voigt = Km*fm + Kf*f
        G_voigt = Gm*fm + Gf*f
        
        return complete_behavior({'G':G_voigt , 'K':K_voigt})
    
    def compute_behavior_ellipsoids(self, Cm, inclusion_behaviors):
    
        Cm = Cm['C']   

        # Computation of fm, volumic fraction of matrix
        fm = 1
        for i in range(len(inclusion_behaviors)) :   
            fm -= inclusion_behaviors[i][1]
            
        # Computation of homogenised behavior
        Ch = fm*Cm
        for i in range(len(inclusion_behaviors)) : 
            Ch += inclusion_behaviors[i][1]*inclusion_behaviors[i][0]['C']
        Sh = inv(Ch)

        return {'C' : Ch, 'S' : Sh}

class Reuss_Bound(Model) : 
    
     def __init__(self):
        """
        Definition of model hypotheses.
        """
        self.type_inclusion = 1 # 1 = Model accepts ellipsoidal and spherical inclusion
        self.behavior_condition = ['anisotropic', 'isotropic']  # The model can be applied to microstructures whose inclusions and matrix are isotropic and anisotropic
        self.n_inclusions = 10 # Max number of different types of inclusions
        self.interphase = False # True if the model works on inclusions with interphase
        self.name = "Reuss Bound"
    
     def compute_behavior(self, Cm, inclusion_behaviors):
        
        Cf, f, ratio = inclusion_behaviors[0]
        fm = 1-f
        
        Km,Gm = Cm["K"], Cm["G"]
        Kf,Gf=Cf["K"],Cf["G"]

        K_reuss = 1/(fm/Km + f/Kf )
        G_reuss = 1/(fm/Gm + f/Gf )
        
        return complete_behavior({'G':G_reuss , 'K':K_reuss})
        
     def compute_behavior_ellipsoids(self, Cm, inclusion_behaviors):
    
        Sm = Cm['S']

        # Computation of fm, volumic fraction of matrix
        fm = 1
        for i in range(len(inclusion_behaviors)) :   
            fm -= inclusion_behaviors[i][1]
        
        # Computation of homogenised behavior
        Sh = fm*Sm
        for i in range(len(inclusion_behaviors)) : 
            Sh += inclusion_behaviors[i][1]*inclusion_behaviors[i][0]['S']
        Ch = inv(Sh)

        return {'C' : Ch, 'S' : Sh}
                    
        
class Mori_Tanaka(Model):
    """
   Mori-Tanaka model. Contains:
    - A description function of the model
    - A function that returns the homogenized behavior of the microstructure.
    """
    
    def __init__(self):
        """
        Definition of model hypotheses.
        """
        self.type_inclusion = 1 # 1 = Model accepts ellipsoidal and spherical inclusion
        self.behavior_condition = ['anisotropic', 'isotropic']  # The model is applied to microstructures whose inclusions and matrix are isotropic.
        self.n_inclusions = 10 # Max number of different types of inclusions
        self.interphase = False # True if the model works on inclusions with interphase
        self.name = "Mori-Tanaka"
    
    def compute_behavior(self, Cm, inclusion_behaviors):
        """
        Calculates the equivalent homogeneous elastic behaviour. 
        Returns a dictionnary of behavior.
        Cm: (dict), dictionary of matrix behavior
        inclusion_behaviors(list), format [(Cf, f, aspect_ratio)] with Cf the dictionaries of inclusion behaviour, and aspect_ratio (a tuple with the two shape ratio values)
        """
        # Retrieving matrix behavior
        Km = Cm['K']
        Gm = Cm['G']
        # Retrieving inclusion behavior
        Cf, f, ratio = inclusion_behaviors[0]
        Kf = Cf['K']
        Gf = Cf['G']
        # Computation of Gh
        denominator = 5*Gm*(3*Km+4*Gm)+6*(1-f)*(Gf-Gm)*(Km+2*Gm)
        numerator = 5*f*Gm*(Gf-Gm)*(3*Km+4*Gm)
        Gh = Gm + numerator/denominator
        # Computation of Kh
        denominator = 3*Km+4*Gm+3*(1-f)*(Kf-Km)
        numerator = f*(Kf-Km)*(3*Km+4*Gm)
        Kh = Km + numerator/denominator
        return {'K': Kh, 'G': Gh}    
    
    def compute_behavior_ellipsoids(self, Cm, inclusion_behaviors):
    
        """
        Calculates the equivalent homogeneous elastic behaviour. 
        Returns a dictionnary of behavior.
        Cm: (dict), dictionary of matrix behavior
        inclusion_behaviors(list), format [(Cf, f, aspect_ratio)] with Cf the dictionaries of inclusion behaviour, and aspect_ratio (a tuple with the two shape ratio values)
        Ch = [Sum(fi*Ci*Ai)+(1-f)*Cm] * T   with Ai = [I+Esh*Sm*(Ci-Cm)]**-1  and T = [Sum(fi*Ai)+(1-f)*I)]**-1
        """
        
        # Number of ellipsoid orientations to satisfy to isotropy
        # recommended above 100
        n_orientation = 500
        
        Sm = Cm['S']
        Cm = Cm['C']
        Id = np.identity(6) 
        
        # Computation of matrix volumic fraction
        fm = 1
        for i in range(len(inclusion_behaviors)) :   
            fm -= inclusion_behaviors[i][1]

        # Computation and storage of every rotation matrices
        Rotation_Matrix = Rotation_matrices(n_orientation)
        
        T = fm*Id       # T = [Sum(fi*Ai)+fm*I)]**-1 is calculated progressively in the loop, inversed in the end
        
        W = np.zeros((6,6)) # Matrix Sum(f*Ci:Ai) in which the contribution of every orientation of every inclusion is added
        
        # Addition of the contribution of each type of inclusion
        for i in range(len(inclusion_behaviors)) :   
            Sfi = inclusion_behaviors[i][0]['S']
            Cfi = inclusion_behaviors[i][0]['C']
            fi = inclusion_behaviors[i][1]
            Ai = (1,inclusion_behaviors[i][2][0],inclusion_behaviors[i][2][0])      # Aspect ratio of the ellipsoids : Ai = a2/a1, a3/a1
        
            fi_1_orientation = fi/n_orientation 
            Esh = Fast_Eshelby_tensor(Ai,Cm,Sm)                     #Eshelby tensor for inclusion i
            Aeshi = inv(Id + np.matmul(Esh,np.matmul(Sm,Cfi-Cm)))   #Localisation tensor for inclusion i
            T += fi*Aeshi                                           #T = [Sum(fi*Ai)+fm*I)]**-1 is calculated progressively in the loop, inversed in the end
            V6i = np.matmul(Cfi,Aeshi)                              #V6i = fi*Ci*Ai in the inclusion coordinate system
            clear_matrix2(V6i)
            # Addition of the contribution of each orientation for 1 type of inclusion
            V3 = Comp66_to_3333(V6i)
            V3L = Matrix_to_vecteur(V3)
            for j in range(n_orientation) :                 
                V3RL = fast_tensor_rotation(V3L,Rotation_Matrix[j])
                V3R = Vecteur_to_matrix(V3RL)
                V = Comp3333_to_66(V3R)                             #V = fi*Ci*Ai for an ellipsoid randomly oriented in the matrix coordinate system
                W += fi_1_orientation * V                           #At the end of th 2 loops, W = Sum(fi*Ci*Ai) for N ellipsoids randomly oriented in the matrix coordinate system

        Ch = np.matmul( (W + fm*Cm) , inv(T))
        Sh = inv(Ch)
        
        return {'C' : Ch, 'S' : Sh}



class Differential_Scheme(Model):
    """
    Differential scheme
    """
    
    def __init__(self):
        """
        Definition of model hypotheses.
        """
        self.type_inclusion = 1 # 1 = Model accepts ellipsoidal and spherical inclusion
        self.behavior_condition = ['anisotropic','isotropic'] # Model accepts anisotropic and isotropic behavior in inclusions
        self.n_inclusions = 10 # Max number of different types of inclusions 
        self.interphase = False
        self.name = "Differential"
    
    ## Useful functions to compute homogenized behavior
    
    def deriv(module, f):
        """
        Function that computes the derivatives of the parameters K and G in relation to the volume fraction of inclusion. Designed to be called by the odeint function during numerical integration.
        module: list, contains the real and imaginary values of the current K, G parameters as well as Kf and Gf specific to the inclusion.
        f: float, current inclusion volume fraction.
        """
        K1, K2, G1, G2, Kf1, Kf2, Gf1, Gf2 = module
        # Creation of complex parameters
        K = K1 + K2*1j
        G = G1 + G2*1j
        Kf = Kf1 + Kf2*1j
        Gf = Gf1 + Gf2*1j
        nu = (3*K-2*G)/(6*K+2*G)
        # Computation of dK
        numerator = K-Kf
        denominator = (1-f)*(1+(Kf-K)/(K+4*G/3))
        dK = -numerator/denominator
        dK1, dK2 = dK.real, dK.imag
        # Computation of dG
        numerator = 15*(1-nu)*(G-Gf)
        denominator = (1-f)*(7-5*nu+2*(4-5*nu)*Gf/G)
        dG = -numerator/denominator
        dG1, dG2 = dG.real, dG.imag
        
        return np.array([dK1, dK2 ,dG1, dG2] + 4*[0])
    
    def compute_behavior(self, Cm, inclusion_behaviors):
        """
        Computes the equivalent homogenized behavior of the microstructure. Returns a dict with the calculated parameters.
        """
        # Retrieving matrix behavior
        Km = Cm['K']
        Gm = Cm['G']
        # Retrieving inclusion behavior
        Cf, f_final, ratio = inclusion_behaviors[0]
        Kf = Cf['K']
        Gf = Cf['G']
        
        # Initialisation of integration parameters
        npoints = int(f_final*100) + 2 # +2 is added for volume fraction < 1% and possible linespace just below
        f = np.linspace(0, f_final, npoints) # List of dilute volumic fractions
        initial_module = []
        for parameter in [Km, Gm, Kf, Gf]:
            initial_module += [parameter.real, parameter.imag]
        initial_module = np.array(initial_module)
        
        # Integration of differential equation
        module = odeint(Differential_Scheme.deriv, initial_module, f)
        
        # Retrieving final result
        final_module = module[-1]
        Kh1, Kh2, Gh1, Gh2 = final_module[:4]  
        # Return result
        return {'K': Kh1+1j*Kh2, 'G': Gh1+1j*Gh2}
    
    def compute_behavior_ellipsoids(self, Cm, inclusion_behaviors):
    
        """
        Calculates the equivalent homogeneous elastic behaviour. 
        Returns a dictionnary of behavior.
        Cm: (dict), dictionary of matrix behavior
        inclusion_behaviors(list), format [(Cf, f, aspect_ratio)] with Cf the dictionaries of inclusion behaviour, and aspect_ratio (a tuple with the two shape ratio values)
        Ch (f+df) = Ch(f) + 1/fm * Sum(dfi*(Ci-Ch)*Aeshi)  with Aeshi = [I+Esh*Sm*(Ci-Cm)]**-1
        """
        
        # Number of ellipsoid orientations to satisfy to isotropy
        # recommended above 100
        n_orientation = 500     
        
        
        Sm = Cm['S']
        Cm = Cm['C']
        Id = np.eye(6) 

        # Creation and storage of every rotation matrices
        Rotation_Matrix = Rotation_matrices(n_orientation)
        
        # Initialisation of dilute solution : Ch is the current concentration at a fixed step
        Ch = Cm
        Sh = Sm        
        
        # Max of volume fraction of fillers
        f_final=0
        
        # Preliminary calculation :
        Esh = []
        f = []
        Cf = []
        for i in range (len(inclusion_behaviors)) :
            Cfi = inclusion_behaviors[i][0]['C']        
            fi = inclusion_behaviors[i][1]              
            Ai = (1,inclusion_behaviors[i][2][0],inclusion_behaviors[i][2][0])     
            f_final=max(fi,f_final)
            
            Esh.append(Fast_Eshelby_tensor(Ai,Cm,Sm))   # List of Eshelby Tensor of each inclusion
            f.append(fi)                               # List of final volumic fraction of each inclusion
            Cf.append(Cfi)                              # List of inclusion Stiffness matrices
        
        # Define the number of steps to reach finale volume fraction of fillers
        n_step = int(f_final*100)+2 # +2 is added for volume fraction < 1% 
        
        # For every step of volumic fraction, fi/n_step is added to the current matrix
        for k in range (1,n_step+1) :             
            dCh = np.zeros((6,6))           # Initialisation of dCh = Sum(dfi*(Ci-Ch)*Aeshi)
            fm_step = 1
            for i in range(len(inclusion_behaviors)) :
                DCi = Cf[i]-Ch
                Aeshi = inv(Id + np.matmul(Esh[i],np.matmul(Sh,DCi)))                
                fi_step = (f[i]/n_step)*k    
                fm_step -= fi_step          # computation of fm at this step               
                
                Pi=np.matmul(DCi,Aeshi)     # Pi = (Ci-Ch)*Aeshi in the inclusion coordinate system
                
                # The contribution of very orientation is added 
                Pi3 = Comp66_to_3333(Pi)            # Pi3 = Pi transformed into 3x3x3x3 Behavior tensor
                Pi3L = Matrix_to_vecteur(Pi3)       # Transformation into a vect to fasten the computation into the following loop
                for j in range(n_orientation) :                                    
                    Pi3RL = fast_tensor_rotation(Pi3L,Rotation_Matrix[j])     # Random Rotation of Pi3             
                    Pi3R = Vecteur_to_matrix(Pi3RL)                           # Pi3 transformed back into a tensor 3x3x3x3
                    Pi6 = Comp3333_to_66(Pi3R)                                # Pi3 transformed into a matrix behavior 6x6
                    dCh += (f[i]/n_step)/n_orientation * Pi6                      # dCh = Sum(dfi*(Ci-Ch)*Aeshi) in the matrix coordinate with inclusion randomly oriented

            Ch = Ch + 1/fm_step*dCh                 # Ch (f+df) = Ch(f) + 1/fm * Sum(dfi*(Ci-Ch)*Aeshi)
            Sh = inv(Ch)
            
            E = isotropic_young_C(Ch)
            nu = isotropic_nu_C(Ch)


        return {'C' : Ch, 'S' : Sh}
    
    

class Self_Consistent_Hill(Model):
    """
    Self-Consistent Model
    """
    def __init__(self):
        """
        Definition of model hypotheses.
        """
        self.type_inclusion = 1 # 1 = Model accepts ellipsoidal and spherical inclusion
        self.behavior_condition = ['isotropic']
        self.n_inclusions = 10 # Max number of different types of inclusions
        self.interphase = False 
        self.name = "Self-consistent"
        self.precision = 10**-2 ## Criterium of convergence of fixed-point algorithm
        self.n_step = 1 # Number of steps to reach final volume fraction
        self.divergence_threshold = 100 # Number of loops in fixed-point algorithme before the model is considered divergent
    
    def Reccurence(Module,f):
        K,G,Km,Gm,Kf,Gf = Module
        ## Compute Kn+1
        numerator = f*(Kf-Km)*(3*K+4*G)
        denominator = 3*Kf+4*G
        nextK = Km + numerator/denominator
        ## Compute Gn+1
        numerator = 5*f*G*(Gf-Gm)*(3*K+4*G)
        denominator = 3*K*(3*G+2*Gf)+4*G*(3*Gf+2*G)        
        nextG = Gm + numerator/denominator
        return nextK,nextG
    
  
    def compute_behavior(self, Cm, inclusion_behaviors):
        # Retrieving matrix behavior
        Km = Cm['K']
        Gm = Cm['G']
        # Retrieving inclusion behavior
        Cf, f, ratio = inclusion_behaviors[0]
        Kf = Cf['K']
        Gf = Cf['G']
        self.n_step = int(f*100)+2 # Number of steps to reach final volume fraction 
        F = np.linspace(0,f,self.n_step) # List of volumic fraction to reach the target
        
        Kinit = Km
        Ginit = Gm
        # Loop to reach the target volumic fraction
        for i in range(len(F)) : 
            fi = F[i]
            #Initialisation of divergence control
            n_loop = 0
            # Initialisation of fixed-point algorithm
            K = Kinit
            G = Ginit
            # Fixed-point algorithm
            precision = self.precision
            nextK,nextG=Self_Consistent_Hill.Reccurence([K,G,Km,Gm,Kf,Gf],fi)
            while abs((nextK-K)/K) > precision or abs((nextG-G)/G) > precision :
                K,G=nextK,nextG
                nextK,NextG=Self_Consistent_Hill.Reccurence([K,G,Km,Gm,Kf,Gf],fi)  
                
                # Stop the computation in case of divergence 
                n_loop += 1   
                if n_loop >self.divergence_threshold : 
                    raise NameError('Self-Consistent model diverge for the values prescribed from the step '+str(i))
                    
            # Updating initialisation
            Kinit = nextK
            Ginit = nextG
        return {'K': nextK, 'G': nextG}
    
    def compute_behavior_ellipsoids(self, Cm, inclusion_behaviors):
        '''
        Calculates the equivalent homogeneous elastic behaviour. 
        Returns a dictionnary of behavior.
        Cm: (dict), dictionary of matrix behavior
        inclusion_behaviors(list), format [(Cf, f, aspect_ratio)] with Cf the dictionaries of inclusion behaviour, and aspect_ratio (a tuple with the two shape ratio values)
        Ch = Cm + Sum{fi*(Cf-Cm)*Aeshi}  with Aeshi = [I+Esh*Sh*(Ci-Ch)]**-1 AND initialisation at the result at f = f-df : it's the reason of the loop over f
        '''
        # Number of ellipsoid orientations to satisfy to isotropy
        # recommended above 100
        n_orientation = 500
        precision = 10**-2  # desired precision in fixed-point algorithm
        Sm = Cm['S']
        Cm = Cm['C']
        Id = np.identity(6) 
        n_inclusions = len(inclusion_behaviors)
    
        # Création des matrices de rotations
        Rotation_Matrix = Rotation_matrices(n_orientation)
    
        # Initialisation of fixed-point algorithm
        Cp = Cm
        Sp = Sm
        
        # Define the number of steps to reach the finale volume fraction of fillers
        fmax = 0.0 
        for j in range(n_inclusions) : 
            fid = inclusion_behaviors[j][1]
            fmax = max(fid,fmax)
        
        self.n_step = int(fmax*100)+1 # +1 is added for volume fraction < 1%
        
        # Loop over volumic fraction to reach the target
        for i in range(self.n_step+1) :
            
            # Fixed-point algorithm
            convergence = 2 
            n_loop = 0
            Eh = isotropic_young(Sp)
            nuh = isotropic_nu(Sp)
            while convergence>precision : 
                
                # Stop the computation in case of divergence 
                n_loop += 1   
                if n_loop >self.divergence_threshold : 
                    raise NameError('Self-Consistent model diverge for the values prescribed from the step '+str(i))
                    
                W = np.zeros((6,6))           # Matrice des contributions de l'inclusion dans Ch
    
                # Addition of the contribution of every type of inclusion
                for j in range(n_inclusions) : 
                    Cf = inclusion_behaviors[j][0]['C']
                    fi_pas = inclusion_behaviors[j][1]*i/self.n_step
                    fi_1_renfort = fi_pas/n_orientation 
                    a2,a3 = inclusion_behaviors[j][2]
                    A = 1,a2,a3
                    
                    Esh = Fast_Eshelby_tensor(A,Cp,Sp)                       # Eshelby tensor of the current homogenous material
                    Aesh = inv(Id + np.matmul(Esh,np.matmul(Sp,Cf-Cp))) # Localisation tensor of the current inclusion in the homogenous matrix
                    
                    V6 = np.dot(Cf-Cm,Aesh)                             # V6 = (Cf-Cm)*Aeshi in the inclusion coordinates
                    V3 = Comp66_to_3333(V6)                             # V3 = V6 transformed in a 3x3x3x3 behavior tensor
                        
                    # Addition of the contribution of every orientation of this type of inclusion
                    V3L = Matrix_to_vecteur(V3)                         # Transformation into vect to fasten computation
                    for k in range(n_orientation) :                 
                        V3RL = fast_tensor_rotation(V3L,Rotation_Matrix[k]) # Rotation into a random orientation
                        V3R = Vecteur_to_matrix(V3RL)                       
                        V = Comp3333_to_66(V3R)                         # V = (Cf-Cm)*Aeshi randomly oriented in the matrix coordinates
                        W += fi_1_renfort * V                           # W = Sum{fi*(Cf-Cm)*Aeshi} with each type of inclusion randomly oriented n_orientation times
                Ch = Cm + W                                             # Ch = Cm + Sum{fi*(Cf-Cm)*Aeshi}
    
                # Update homogenised material
                Cp = Ch
    
                # Convergence update (end of loop if convergence small enough)
                E = isotropic_young_C(Cp)
                nu = isotropic_nu_C(Cp)
                convergence = abs((E-Eh)/Eh)+abs((nu-nuh)/nuh)
                
                Eh = E
                nuh = nu           
    
                # Transformation of material in isotropic matrice
                Sp = Isotropic_Compliance_Matrix(Eh,nuh)
                Cp = inv(Sp)
                # next step of volumic fraction
    
        return {'C' : Cp, 'S' : Sp}
    
class Self_Consistent_III(Model):
    """
    Hypotheses : 
    -isotropic
    -spherical reinforcements 
    -elastic deformations 
    Self-coherent model. Contains :
    - A function that checks if the model is applicable to a given structure.
    - A model description function (TODO: write a function that returns a description of the model as a str and that could be called in the main)
    - A function that returns the homogenized behavior of the microstructure.
    - Functions that compute a particular characteristic (volume fraction of an inclusion, radius of an inclusion, behavior of an inclusion, etc.) from a target homogenized behavior (TODO).
    """
    
    def __init__(self):
        """
        Definition of model hypotheses.
        """
        self.type_inclusion = 0 # 0 = Model accepts only spherical inclusion
        self.behavior_condition = ['isotropic']  # Model accepts only isotropic behavior in matrix and inclusion
        self.n_inclusions = 1 # Model accepts only one type of inclusion
        self.interphase = False 
        self.name = "Generalised self-consistent"
  
    def compute_behavior(self, Cm, inclusion_behaviors):
        """
        Computes the equivalent homogenized behavior of the microstructure. Returns a dict with the calculated parameters.
        """
        # Retrieving matrix behavior
        Km, Gm, num = Cm['K'], Cm['G'], Cm['nu']
        # Retrieving inclusion behavior
        Cf, f, ratio = inclusion_behaviors[0]
        Kf, Gf, nuf = Cf['K'], Cf['G'], Cf['nu']

        ## Useful constant to compute G         
        dm=(Gf/Gm)-1 
        eta1=dm*(49-50*nuf*num)+35*(dm+1)*(nuf-2*num)+35*(2*nuf-num) 
        eta2=5*nuf*(dm-7)+7*(dm+5) 
        eta3=(dm+1)*(8-10*num)+(7-5*num) 
        
        A=8*f**(10/3)*eta1*dm*(4-5*num)-2*f**(7/3)*(63*dm*eta2+2*eta1*eta3)+252*dm*eta2*f**(5/3)-50*dm*(7-12*num+8*num**2)*eta2*f+4*(7-10*num)*eta2*eta3 
        B=-4*dm*(1-5*num)*eta1*f**(10/3)+4*(63*dm*eta2+2*eta1*eta3)*f**(7/3)-504*dm*eta2*f**(5/3)+150*dm*(3-num)*num*eta2*f+3*(15*num-7)*eta2*eta3 
        D=4*dm*(5*num-7)*eta1*f**(10/3)-2*f**(7/3)*(63*dm*eta2+2*eta1*eta3)+252*dm*eta2*f**(5/3)+25*dm*(num**2-7)*eta2*f-(7+5*num)*eta2*eta3 
        
        ## Computation of G
        delta=B*B-4*D*A 
        sol1=(-B - delta**(1/2))/(2*A) 
        sol2=(-B + delta**(1/2))/(2*A) 
        sol=sol1 
        if ((sol1.real)<0) : 
            sol=sol2
            
        Gh=sol*Gm
        Kh=Km+f*(Kf-Km)/(1+(1-f)*(Kf-Km)/(Km+(4/3)*Gm))
        
        return {'K': Kh, 'G': Gh}

class Self_Consistent_IV(Model):
    """
    Hypotheses : 
    Assumptions: 
    -isotropic
    -Spherical reinforcements 
    -elastic deformations 

    Self-consistent model. Contains :
    - A function that checks if the model is applicable to a microstructure.
    - A model description function 
    - A function that returns the homogenized behavior of the microstructure.
    - Functions that computes a particular characteristic (volume fraction of an inclusion, radius of an inclusion, behavior of an inclusion, etc.) from a target homogenized behavior 
    """
    def __init__(self, R_inclusion=1):
        """
       Definition of model hypotheses.
        """
        self.type_inclusion = 0                  # 0 = Model accepts only spherical inclusion
        self.behavior_condition = ['isotropic']  # Model accepts only isotropic behavior in matrix and inclusion
        self.n_inclusions = 1                    # Model accepts only one type of inclusion
        self.interphase = True 
        self.R_inclusion = R_inclusion
        self.name = "4-phases self-consistent"

    def compute_behavior(self, Cm, inclusion_behaviors):
        """
        Calculates the equivalent homogenized behavior of the microstructure. Returns a dict with the calculated parameters. Currently, only calculates the shear modulus.
        TODO: complete with the complete calculation (K and G)
        """
        # Retrieving materials parameters
        Cf, f, ratio0 = inclusion_behaviors[0]
        Cv, cf, ratio1 = inclusion_behaviors[1]
        Km,Gm,num = Cm['K'], Cm['G'], Cm['nu']
        Kf,Gf,nuf = Cf['K'], Cf['G'], Cf['nu']
        Kv,Gv,nuv = Cv['K'], Cv['G'], Cv['nu']
        
        Rf = self.R_inclusion
        Rm = Rf/(f**(1/3))
        Rv = Rm*(f+cf)**(1/3)
        
        
        a1=(Gf/Gv)*(7+5*nuf)*(7-10*nuv)-(7-10*nuf)*(7+5*nuv) 
        b1=4*(7-10*nuf)+(Gf/Gv)*(7+5*nuf) 
        c1=(7-5*nuv)+2*(Gf/Gv)*(4-5*nuv) 
        d1=(7+5*nuv)+4*(Gf/Gv)*(7-10*nuv) 
        e1=2*(4-5*nuf)+(Gf/Gv)*(7-5*nuf) 
        f1=(4-5*nuf)*(7-5*nuv)-(Gf/Gv)*(4-5*nuv)*(7-5*nuf) 
        alpha1=(Gf/Gv)-1 
        
        a2=(Gv/Gm)*(7+5*nuv)*(7-10*num)-(7-10*nuv)*(7+5*num) 
        b2=4*(7-10*nuv)+(Gv/Gm)*(7+5*nuv) 
        c2=(7-5*num)+2*(Gv/Gm)*(4-5*num) 
        d2=(7+5*num)+4*(Gv/Gm)*(7-10*num) 
        e2=2*(4-5*nuv)+(Gv/Gm)*(7-5*nuv) 
        f2=(4-5*nuv)*(7-5*num)-(Gv/Gm)*(4-5*num)*(7-5*nuv) 
        alpha2=(Gv/Gm)-1 
    
        M1=np.zeros(shape=(4,4),dtype=complex)
        M1[0,0]=(5*(1-nuv))**(-1)*c1/3 
        M1[0,1]=(5*(1-nuv))**(-1)*Rf**2*(3*b1-7*c1)/(5*(1-2*nuf)) 
        M1[0,2]=(5*(1-nuv))**(-1)*(-12*alpha1/Rf**5) 
        M1[0,3]=(5*(1-nuv))**(-1)*4*(f1-27*alpha1)/(15*(1-2*nuf)*Rf**3) 
        M1[1,0]=0 
        M1[1,1]=(5*(1-nuv))**(-1)*(1-2*nuv)*b1/(7*(1-2*nuf)) 
        M1[1,2]=(5*(1-nuv))**(-1)*(-20*(1-2*nuv)*alpha1)/(7*Rf**7) 
        M1[1,3]=(5*(1-nuv))**(-1)*(-12*alpha1*(1-2*nuv))/(7*(1-2*nuf)*Rf**5) 
        M1[2,0]=(5*(1-nuv))**(-1)*Rf**5*alpha1/2 
        M1[2,1]=(5*(1-nuv))**(-1)*(-Rf**7*(2*a1+147*alpha1))/(70*(1-2*nuf)) 
        M1[2,2]=(5*(1-nuv))**(-1)*d1/7 
        M1[2,3]=(5*(1-nuv))**(-1)*Rf**2*(105*(1-nuv)+12*alpha1*(7-10*nuv)-7*e1)/(35*(1-2*nuf)) 
        M1[3,0]=(5*(1-nuv))**(-1)*(-5/6)*(1-2*nuv)*alpha1*Rf**3 
        M1[3,1]=(5*(1-nuv))**(-1)*7*(1-2*nuv)*alpha1*Rf**5/(2*(1-2*nuf)) 
        M1[3,2]=0 
        M1[3,3]=(5*(1-nuv))**(-1)*e1*(1-2*nuv)/(3*(1-2*nuf)) 
        
        M2=np.zeros(shape=(4,4),dtype=complex)
        M2[0,0]=(5*(1-num))**(-1)*c2/3 
        M2[0,1]=(5*(1-num))**(-1)*Rv**2*(3*b2-7*c2)/(5*(1-2*nuv)) 
        M2[0,2]=(5*(1-num))**(-1)*(-12*alpha2/Rv**5) 
        M2[0,3]=(5*(1-num))**(-1)*4*(f2-27*alpha2)/(15*(1-2*nuv)*Rv**3) 
        M2[1,0]=0 
        M2[1,1]=(5*(1-num))**(-1)*(1-2*num)*b2/(7*(1-2*nuv)) 
        M2[1,2]=(5*(1-num))**(-1)*(-20*(1-2*num)*alpha2)/(7*Rv**7) 
        M2[1,3]=(5*(1-num))**(-1)*(-12*alpha2*(1-2*num))/(7*(1-2*nuv)*Rv**5) 
        M2[2,0]=(5*(1-num))**(-1)*Rv**5*alpha2/2 
        M2[2,1]=(5*(1-num))**(-1)*(-Rv**7*(2*a2+147*alpha2))/(70*(1-2*nuv)) 
        M2[2,2]=(5*(1-num))**(-1)*d2/7 
        M2[2,3]=(5*(1-num))**(-1)*Rv**2*(105*(1-num)+12*alpha2*(7-10*num)-7*e2)/(35*(1-2*nuv)) 
        M2[3,0]=(5*(1-num))**(-1)*(-5/6)*(1-2*num)*alpha2*Rv**3 
        M2[3,1]=(5*(1-num))**(-1)*7*(1-2*num)*alpha2*Rv**5/(2*(1-2*nuv)) 
        M2[3,2]=0 
        M2[3,3]=(5*(1-num))**(-1)*e2*(1-2*num)/(3*(1-2*nuv)) 
        
        P = np.dot(M2,M1) 
        
        Z12 = P[0,0]*P[1,1]-P[1,0]*P[0,1] 
        Z14 = P[0,0]*P[3,1]-P[3,0]*P[0,1] 
        Z42 = P[3,0]*P[1,1]-P[1,0]*P[3,1] 
        Z23 = P[1,0]*P[2,1]-P[2,0]*P[1,1] 
        Z43 = P[3,0]*P[2,1]-P[2,0]*P[3,1] 
        Z13 = P[0,0]*P[2,1]-P[2,0]*P[0,1] 
    
        A = 4*Rm**10*(1-2*num)*(7-10*num)*Z12+20*Rm**7*(7-12*num+8*num**2)*Z42+12*Rm**5*(1-2*num)*(Z14-7*Z23)+20*Rm**3*(1-2*num)**2*Z13+16*(4-5*num)*(1-2*num)*Z43
        B = 3*Rm**10*(1-2*num)*(15*num-7)*Z12+60*Rm**7*(num-3)*num*Z42-24*Rm**5*(1-2*num)*(Z14-7*Z23)-40*Rm**3*(1-2*num)**2*Z13-8*(1-5*num)*(1-2*num)*Z43
        C = -Rm**10*(1-2*num)*(7+5*num)*Z12+10*Rm**7*(7-num**2)*Z42+12*Rm**5*(1-2*num)*(Z14-7*Z23)+20*Rm**3*(1-2*num)**2*Z13-8*(7-5*num)*(1-2*num)*Z43
        
        delta=B*B-4*C*A 
        sol1=(-B - delta**(1/2))/(2*A) 
        sol2=(-B + delta**(1/2))/(2*A) 
        sol=sol2 
        if (sol2.real<0):
            sol=sol1 
     
        Gh=sol*Gm
        X=(3*Km+4*Gm)*(f+cf)*( (Kf-Kv)*f*(3*Km+4*Gv)+(Kv-Km)*(cf+f)*(3*Kf+4*Gv)) 
        Y=3*(Kv-Kf)*f*( (f+cf)*(3 *Km+4*Gv)+4*(Gm-Gv)) + (3*Kf+4*Gv)*(f+cf)*(3*(cf+f)*(Km-Kv)+(3*Kv+4*Gm)) 
        Kh=Km+X/Y
        return {'K': Kh, 'G': Gh}
    
     
 
#%% Useful functions
def bulk_to_young(K, G):
    """
    Transforms  K and G modulus into E and nu
    """
    E = 9*K*G/(3*K+G)
    nu = (3*K-2*G)/(2*(3*K+G))
    return E, nu
   
def young_to_bulk(E, nu):
    """
    Transforms E and nu modulus into K and G 
    """
    K = E/(3*(1-2*nu))
    G = E/(2*(1+nu))
    return K, G   

def bulk_to_shear(K, E):
    """
    Transforms  K and E modulus into G and nu
    """
    G = 3*K*E/(9*K-E)
    nu = (3*K-E)/(6*K)
    return G, nu

def complete_behavior(behavior):
    """
    If the input behaviour is isotropic, completes it with E and nu or K and G.
    Otherwise, completes it with C or S if the input matrix is invertible.
    """
    parameters = list(behavior.keys())
    result = behavior
    nu_max = 0.4999999
    # Case of null (porous media) and incompressible (nu=0.5) media
    for parameter, values in result.items():
        # Isotropic elastic behavior
        if type(values) in [float, int]:
            if values==0:
                result[parameter] = 10**(-12)
            elif values==0.5 and parameter=='nu':
                result[parameter] = nu_max
        # Isotropic visco-elastic behavior
        elif type(values)==np.ndarray and type(values[0])!=np.ndarray:
            for i, value in enumerate(values):
                if value==0:
                    result[parameter][i] = 10**(-12)
                elif value==0.5 and parameter=='nu':
                    result[parameter][i] = nu_max
    # Isotropic K and G
    if parameters[:2]==['K', 'G'] or parameters[:2]==['G', 'K']:
        K, G = behavior['K'], behavior['G']
        E, nu = bulk_to_young(K, G)
        result['E'], result['nu'] = E, nu
    # Isotropic E and nu
    elif parameters[:2]==['E', 'nu'] or parameters[:2]==['nu', 'E']:
        E, nu = behavior['E'], behavior['nu']        
        K, G = young_to_bulk(E, nu)
        result['K'], result['G'] = K, G
    # Isotropic K and E
    elif parameters[:2]==['K', 'E'] or parameters[:2]==['E', 'K']:
        K, E = behavior['K'], behavior['E']        
        G, nu = bulk_to_shear(K, E)
        result['G'], result['nu'] = G, nu
    # Anisotropic
    elif parameters[0]=='C':
        C = behavior['C']
        try:
            S = np.linalg.inv(C)
        except:
            # C non invertible
            S = None
        result['S'] = S
    elif parameters[0]=='S':
        S = behavior['S']
        try:
            C = np.linalg.inv(S)
        except:
            # S non invertible
            C = None
        result['C'] = C
    
    # Return result
    return result


def Isotropic_behavior(behavior) : 
    S = behavior['S']
    E = float(isotropic_young(S))
    nu = float(isotropic_nu(S))
    return complete_behavior({'E':E,'nu':nu})

def display_behavior(behavior):
    """
    Input: behavior dict
    Returns a string with a clean presentation of the behavior.
    """
    matrix_behavior = False
    result = str() # Initialisation
    for parameter, value in behavior.items():
        # Simple values
        if type(value)==float or type(value)==np.float64:
            result += "{}: {:.3f}\n".format(parameter, value)
        # Matrices
        elif type(value)==np.ndarray and np.shape(value)==(6,6):
            matrix_behavior = True
            result += "{}: \n".format(parameter)
            for i in range(6):
                for j in range(6):
                    result += "{:.4e}  ".format(value[i,j])
                result += "\n"
        # Visco-elastic lists
        else:
            result += "{}: Visco-elastic\n".format(parameter)
        
    if matrix_behavior : 
        isotropic_behavior = Isotropic_behavior(behavior)
        for parameter, value in isotropic_behavior.items():
            if type(value)==float:
                result += "{}: {:.4f}\n".format('isotropic_'+parameter, value)
                
    return result



#%% Definition of model, behaviors et inclusion shape 
list_models = [Voigt_Bound, Reuss_Bound, Mori_Tanaka, Differential_Scheme, Self_Consistent_Hill, Self_Consistent_III, Self_Consistent_IV]
dict_behaviors_visco = {'Elastic isotropic (K & G)': ['K', 'G'],
                        'Elastic isotropic (E & nu)': ['E', 'nu'],
                        'Visco-elastic 1': ['K', "G'", "G''"],
                        'Visco-elastic 2': ["K'", "K''", "G'", "G''"],
                        'Visco-elastic 3': ["E'", "E''", 'K']}
dict_behaviors = {'Isotropic (K & G)': ['K', 'G'],
                  'Isotropic (E & nu)': ['E', 'nu'],
                  'Anisotropic (Compliance)': ['S'],
                  'Anisotropic (Stifness)': ['C']}
dict_types = {0: 'Spheres', 1: 'Ellipsoids'}
    
   
    


---

## II- Useful functions

In [7]:
parameters_name = {'K': 'Bulk modulus K',
                   'G': 'Shear modulus G',
                   'E': 'Young modulus E',
                   'nu': "Poisson's ratio " + r'$\nu$',
                   'C': "Stifness matrix C",
                   'S': "Compliance matrix S"}
parameters_name_bis = {value: key for (key, value) in parameters_name.items()}
layout={'border': '1px solid #FF425BF5'}

def gen_tab_behavior(anisotropic=True):
    """
    Generate a 'tab' widget defining the behavior parameters.
    Each generated tab refers to a behavior from the 'dict_behaviors' dictionnary defined in 'classes.py'.
    The 'isotropic' parameter is 'True' if one only wants to display isotropic behaviors.
    The function returns:
    - A list of list of widgets 'list_widgets' with as many lists as tabs. Each list contains the widgets for the parameters associated with the behavior of the corresponding tab.
    - The tab widget to be displayed.
    """
    behaviors_str = list(dict_behaviors.keys()) # Lists the names of the behaviors implemented in classes.py
    list_widgets = [] # List of lists, each list refers to a tab of the final widget and contains the non-formatted widgets of this tab
    tab_titles = [] # Tabs names, each tab refers to a behavior type ('Isotropic K and G', 'Anisotropic', etc..) 
    # Generation of the widgets for each tab
    for behavior_str in behaviors_str:
        if 'Anisotropic' in behavior_str and not anisotropic:
            continue
        widgets_onglet = []
        parameters = dict_behaviors[behavior_str] # Behavior-associated parameters (example : ['K', 'G'] for Isotropic)
        for parameter in parameters:
            w = widgets.BoundedFloatText(value=200, min=0, max=10**12, step=1) # 'parameter' associated widget
            if parameter == 'nu':
                w.max = 0.5
                w.value = 0.3
                w.step = 0.1
            if 'Anisotropic' in behavior_str:
                # Recovery of the anisotropic behaviors files
                input_folder = 'inputs/anisotropic_behaviors'
                behavior_files = listdir(input_folder)
                # Selection of .csv and .txt files
                behavior_files = [file for file in behavior_files if (file.endswith('.txt') or file.endswith('.csv'))]
                # Widget creation
                w = widgets.Dropdown(options=behavior_files)
            w_label = widgets.Label(value=parameters_name[parameter])
            widgets_onglet.append(widgets.HBox([w_label, w]))
        # Input files description
        if 'Anisotropic' in behavior_str:
            w_label = widgets.Label(value="Entry file must be a csv file with 6 rows and columns of numbers.\nInput folder: inputs/anisotropic_behaviors")
            widgets_onglet.append(w_label)
        list_widgets.append(widgets_onglet)
        tab_titles.append(behavior_str)
    # Tab creation
    tab = widgets.Tab()
    tab.children = [widgets.VBox(w) for w in list_widgets]
    for pos, title in enumerate(tab_titles):
        tab.set_title(pos, title)
    return list_widgets, tab

def read_behavior(tab, list_widgets):
    """
    Reads tab widgets created by the 'gen_tab_behavior', returns a 'behavior' dictionnary.
    """
    behavior_int = tab.selected_index # Index of the tab selected by the user
    behavior_str = tab.get_title(behavior_int)
    widgets_parameters = list_widgets[behavior_int] # Parameter widgets of the active tab
    if 'Isotropic' in behavior_str:
        # Isotropic behaviors
        behavior = {parameters_name_bis[w.children[0].value]: w.children[1].value for w in widgets_parameters}
    else:
        # Anisotropic behaviors
        w_parameter = widgets_parameters[0] # HBox widget containing a label widget and a text widget
        parameter = parameters_name_bis[w_parameter.children[0].value] # Name of the parameter
        file_name = "inputs/anisotropic_behaviors/" + w_parameter.children[1].value
        # Anisotropic behavior file reading
        value = []
        with open(file_name) as csvfile:
            reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
            for row in reader: # each row is a list
                value.append(row)
        # Generation of the 'behavior' dict
        behavior = {parameter: np.array(value)}
    return behavior

def gen_tab_type():
    """
    Creates a tab widget setting the inclusion geometric parameters (aspects ratio, orientation, etc..).
    TODO : Inclure l'orientation
    Each tab refers to a key of the 'dict_types' dictionnary defined in 'classes.py'.
    Returns:
    - a list of list of widgets 'list_widgets' with as many lists as tabs. Each list contains widgets associated to a type.
    - a tab widget to be displayed.
    """
    list_widgets = [] # List of lists, each list refers to a tab and contains this tab widgets
    # Tab per tab widgets generation
    for type_int in dict_types.keys():
        if type_int == 0:
            # Spheres, additional parameters are not needed
            list_widgets.append([])
        else:
            # Initialisation
            list_widgets.append([])
            # Ellipsoids
            for n in ['1', '2']:
                w_label = widgets.Label(value="Aspect ratio " + str(n))
                w_aspect_ratio = widgets.BoundedFloatText(value=0.5, min=0.1, max=10000, step=0.5)
                list_widgets[-1] += [widgets.HBox([w_label, w_aspect_ratio])]
    # Tab creation
    tab = widgets.Tab()
    # Tabs attribution
    tab.children = [widgets.HBox(w) for w in list_widgets]
    # Tabs titles attribution
    for pos, title in dict_types.items():
        tab.set_title(pos, title)
    return list_widgets, tab         

def read_type(tab, list_widgets):
    """
    Reads a 'tab' widget generated by the 'gen_tab_type' function, returns the inclusion's type and aspect_ratios parameters.
    """
    type_int = tab.selected_index # Active tab
    tab_name = tab.get_title(type_int) # Active tab name
    if tab_name=='Spheres':
        # Spheres
        aspect_ratio = [1,1]
    elif tab_name=='Ellipsoids':
        # Ellipsoids
        c1 = list_widgets[type_int][0].children[1].value
        c2 = list_widgets[type_int][1].children[1].value
        aspect_ratio = [c1, c2]
    return type_int, aspect_ratio

def str_to_model(model_name):
    """
    Returns the model instance associated to the 'model_name' string. 
    """
    for Model in list_models:
        model = Model()
        if model.name.upper() == model_name.upper():
            return model
        
def incr(value, value_incr, mini, maxi):
    """
    Adds 'value_incr' to the 'value' float if it doesn't exceed mini or maxi.
    Returns a bool stating whether 'value' has been changed.
    """
    result = value
    result += value_incr
    if result > maxi:
        result = maxi
    if result < mini:
        result = mini
    changed = (result != value)
    return result, changed

---

## III- Computation of the homogenized behavior of defined microstructures
This section lets the user manually generate a microstructure, then computes its homogenised behavior using available models.

In [8]:
dict_inclusions = {}
# Initialisation of the list of generated inclusions. 'dict_inclusions' format is {inclusion_name(str): inclusion(Inclusion or InclusionAndInterphase)}

# Inclusion name
w_label = widgets.Label(value='Inclusion name')
n_inclusion = 0 # Index used to automatically give each inclusion a unique name
w_name = widgets.Text(value='inclusion'+str(n_inclusion))

# Inclusion geometric type
widgets_type, tab_type = gen_tab_type() 

# Inclusion behavior
caption = widgets.Label(value='Inclusion behavior')
list_widgets, tab = gen_tab_behavior()

# Inclusion generation
button_generate_inclusion = widgets.Button(description="Generate Inclusion")
output = widgets.Output()
def generate_inclusion(b):
    """
    Called when the 'Generate inclusion' button is toggled, creates an inclusion with the chosen parameters.
    """
    global n_inclusion
    # Recovery of the chosen parameters 
    output.clear_output()
    inclusion_name = w_name.value
    if inclusion_name in list(dict_inclusions.keys()):
        with output:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type, widgets_type)
        behavior = read_behavior(tab, list_widgets)
        inclusion = Inclusion(type_inclusion, behavior, name=inclusion_name, aspect_ratio=inclusion_aspect_ratio)
        dict_inclusions[inclusion_name] = inclusion
        with output:
            print("Inclusion generated: ", inclusion)
        # Automatic update of the inclusion name
        n_inclusion += 1
        w_name.value = 'inclusion'+str(n_inclusion)
    
button_generate_inclusion.on_click(generate_inclusion)

# Inclusion and interphase generation
# Instance name
w_label_bis = widgets.Label(value='Instance name')
n_inclusion_bis = 0 # Index used to automatically give each inclusion a unique nam
w_name_bis = widgets.Text(value='microstructure'+str(n_inclusion_bis))

# Inclusion geometric type
widgets_type_bis, tab_type_bis = gen_tab_type() 

# Inclusion behavior
caption_incl = widgets.Label(value='Inclusion behavior')
list_widgets_incl, tab_incl = gen_tab_behavior()
# Interphase behavior
caption_inter = widgets.Label(value='Interphase behavior')
list_widgets_inter, tab_inter = gen_tab_behavior()

# Instance generation
button_generate_inclusion_bis = widgets.Button(description="Generate inclusion with interphase", layout={'width': 'max-content'})
output_bis = widgets.Output()

def generate_inclusion_bis(b):
    """
    Called when the 'Generate inclusion and interphase' button is toggled, creates an InclusionAndInterphase instance with the chosen parameters.
    """
    global n_inclusion_bis
    # Recovery of the chosen parameters
    output_bis.clear_output()
    instance_name = w_name_bis.value
    if instance_name in list(dict_inclusions.keys()):
        with output_bis:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type_bis, widgets_type_bis)
        behavior_incl = read_behavior(tab_incl, list_widgets_incl) # behavior of the inclusion
        behavior_inter = read_behavior(tab_inter, list_widgets_inter) # behavior of the interphase
        inclusion = Inclusion(type_inclusion, behavior_incl, name=instance_name+'_inclusion', aspect_ratio=inclusion_aspect_ratio)
        interphase = Inclusion(type_inclusion, behavior_inter, name=instance_name+'_interphase', aspect_ratio=inclusion_aspect_ratio)
        instance = InclusionAndInterphase(inclusion, interphase, name=instance_name)
        dict_inclusions[instance_name] = instance
        with output_bis:
            print("Inclusion generated: ", instance)
        # Automatic update of the instance name
        n_inclusion_bis += 1
        w_name_bis.value = 'microstructure'+str(n_inclusion_bis)
    
button_generate_inclusion_bis.on_click(generate_inclusion_bis)

# Microstructure generation
microstructure = None # Initialisation

# Button linked functions
def add_inclusion_to_structure(b):
    """
    Called when the 'Add inclusion' button is toggled.
    Creates a widget linked to the volume fraction of the selected inclusion and adds it to the 'widgets_f' dict.
    Also creates a 'Remove inclusion' button and adds it to the 'buttons' dict.
    Eventually displays the generated widgets.
    """
    out2.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions.value]
    except KeyError:
        return None
    if inclusion in list(widgets_f.keys()):
        with out2:
            print("Already added")
    else:
        w_name = widgets.Label(inclusion.name)
        w_b = widgets.Button(description="Remove inclusion")
        w_b.on_click(remove_inclusion)
        buttons_suppress[w_b] = inclusion
        # Inclusion and interphase
        if type(inclusion)==InclusionAndInterphase:
            w_f_incl = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f inclusion')
            w_f_inter = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f interphase')
            widgets_f[inclusion] = (w_name, w_f_incl, w_f_inter)
            with out1:
                display(w_name, widgets.HBox([w_f_incl, w_f_inter, w_b]))
        # Simple inclusion
        else:
            w_f = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f')
            widgets_f[inclusion] = (w_name, w_f)
            with out1:
                display(w_name, widgets.HBox([w_f, w_b]))

def add_inclusion_to_list(b):
    """
    Called when an inclusion or inclusion and interphase is generated.
    Updates the inclusion selection widgets.
    """
    w_inclusions.options = list(dict_inclusions.keys())
    w_inclusions_info.options = list(dict_inclusions.keys())
    
    
def remove_inclusion(b):
    """
    Called when an inclusion is removed from the structure.
    Recovers the selected inclusion, closes its widgets and removes it from the 'widgets_f' dict.
    """
    out2.clear_output()
    inclusion = buttons_suppress[b]
    # Closing widgets
    b.close()
    for w in widgets_f[inclusion]:
        w.close()
    del widgets_f[inclusion]
    del buttons_suppress[b]

def generate_microstructure(b):
    """
    Generates a microstructure with the parameters set by the user.
    Displays an error message if the chosen volume fractions are not consistent.
    Eventually displays a description of the generated microstructure.
    """
    global microstructure
    matrix_behavior = read_behavior(tab_m, widgets_m) # Reading the 'Matrix beahvior' widgets
    dict_inclusions = {}
    # Reading the chosen volume fractions
    for inclusion, widgets in widgets_f.items():
        # Inclusions with an interphase
        if type(inclusion)==InclusionAndInterphase:
            w_name, w_f_incl, w_f_inter = widgets
            f = [w_f_incl.value, w_f_inter.value]
        else:
            w_name, w_f = widgets
            f = w_f.value
        dict_inclusions[inclusion] = f
    # Microstructure generation
    out3.clear_output()
    try:
        microstructure = Microstructure(matrix_behavior, dict_inclusions)
        with out3:
            print("Microstructure generated\n" + str(microstructure))
            # Microstructure 3D representation
            microstructure.draw()
    except NameError:
        microstructure = None
        with out3:
            print("Inconsistent choice of volume fractions")

### Displaying inclusion widgets
# Simple inclusion
display(Markdown("## Inclusion generation"))
w_inclusion = widgets.VBox([w_label, w_name, widgets.Label(value='Inclusion type'), tab_type, caption, tab, button_generate_inclusion, output],
                          layout=layout)
display(w_inclusion)
# Inclusion with an interphase
display(Markdown("## Inclusion and interphase generation"))
w_inclusion_bis = widgets.VBox([w_label_bis,
                                w_name_bis,
                                widgets.Label(value='Inclusion type'),
                                tab_type_bis,
                                caption_incl,
                                tab_incl,
                                caption_inter,
                                tab_inter,
                                button_generate_inclusion_bis,
                                output_bis],
                          layout=layout)
display(w_inclusion_bis)

### Inclusion info and deletion
w_label = widgets.Label(value="Displays info on the generated inclusions")
w_inclusions_info = widgets.Dropdown(options=list(dict_inclusions.keys()), layout={'width': 'max-content'})
w_delete = widgets.Button(description="Delete inclusion", layout={'width': 'max-content'})
out_inclusions_info = widgets.Output() # Displays a description of the selected inclusion

def display_info(change):
    """
    Called when a different inclusion is selected on the 'w_inclusions_info' widget.
    Recovers the selected inclusion and displays its description.
    """
    out_inclusions_info.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions_info.value]
    except KeyError:
        inclusion = None
    with out_inclusions_info:
        print(inclusion)
        
w_inclusions_info.observe(display_info, names='value')
# Inclusions deletion
def delete_inclusion(b):
    """
    Called when an inclusion is deleted.
    Recovers the selected inclusion and deletes it from the generated inclusions dict.
    """
    inclusion_name = w_inclusions_info.value
    try:
        del dict_inclusions[inclusion_name]
    except KeyError:
        None
    w_inclusions.options = list(dict_inclusions.keys())
    w_inclusions_info.options = list(dict_inclusions.keys())

w_delete.on_click(delete_inclusion)
# Displaying inclusion info widgets
display(Markdown("## Inclusions info"))
display(widgets.VBox([w_label, widgets.HBox([w_inclusions_info, w_delete]), out_inclusions_info], layout=layout))

# Matrix behavior
caption = widgets.Label(value='Matrix behavior')
widgets_m, tab_m = gen_tab_behavior(anisotropic=False)

# Add inclusion to structure
w_inclusions = widgets.Dropdown(options=list(dict_inclusions.keys()), layout={'width': 'max-content'})
button_add_inclusion = widgets.Button(description="Add inclusion")
out1 = widgets.Output()
out2 = widgets.Output()
widgets_f = {} # Dict of the added inclusions and their widgets ('name','volume fraction')
buttons_suppress = {} # Dict of the 'Remove inclusion' buttons and their inclusions

button_add_inclusion.on_click(add_inclusion_to_structure)
button_generate_inclusion.on_click(add_inclusion_to_list)
button_generate_inclusion_bis.on_click(add_inclusion_to_list)

# Microstructure generation
b_generate_structure = widgets.Button(description='Generate microstructure', layout={'width': 'max-content'})
# TODO : widget 'valid' qui indique en temps réel si les fractions volumiques choisies sont cohérentes
out3 = widgets.Output()
b_generate_structure.on_click(generate_microstructure)

# Displays the microstructure widgets
display(Markdown("## Microstructure generation"))
w_micro = widgets.VBox([caption, tab_m, widgets.HBox([w_inclusions, button_add_inclusion, out2]), out1, b_generate_structure, out3],
                      layout=layout)
display(w_micro)

# Model selection
def test_models(b=None):
    """
    Called when a microstructure is generated.
    Tests the implemented models on the generated microstructure and creates a 'valid_models' list of compatible models.
    """
    valid_models = []
    if microstructure == None:
        # Checks whether the microstructure has been generated
        return None
    for Model in list_models:
        model = Model()
        valid = model.check_hypothesis(microstructure)
        if valid:
            # The microstructure fits the model hypothesis
            valid_models.append((model.name, model))
    # Updates the model selection widget
    select_model.options = valid_models

valid_models = [] # List of the microstructure-compatible models: [(model_name, Model)]
select_model = widgets.Dropdown()
test_models()
label = widgets.Label(value="Select a model. Only compatible models will be displayed.")
b_compute = widgets.Button(description='Compute behavior')
output_behavior = widgets.Output()

def compute_model(b):
    """
    Called when the 'Compute behavior' button is toggled.
    Recovers the selected model, computes the microstructure homogenised behavior and displays it.
    """
    output_behavior.clear_output()
    with output_behavior:
        print("Computing ...")
    model = select_model.value
    homogenised_behavior = model.compute_h_behavior(microstructure)
    output_behavior.clear_output()
    with output_behavior:
        print("Homogenized behavior - {} model".format(model.name))
        print(display_behavior(homogenised_behavior))
        if microstructure.check_Hashin_hypothesis() : 
            print("Hashin-Shtrikman bounds")
            print(microstructure.Hashin_bounds())
    

b_generate_structure.on_click(test_models)
b_compute.on_click(compute_model)

# Displays the model widgets
display(Markdown("## Available models"))
w_model = widgets.VBox([label, widgets.HBox([select_model, b_compute]), output_behavior],
                      layout=layout)
display(w_model)

## Inclusion generation

## Inclusion and interphase generation

## Inclusions info

## Microstructure generation

## Available models

### Model comparison

In [9]:
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation) # Disable the matplotlib warning message
matplotlib.rcParams.update({'font.size': 6}) # Sets the font size in matplotlib graphs

def draw_all_data(subplots):
    """
    Draws a graph with the data contained in the 'subplots' database.
    The database format is defined below.
    """
    global fig
    out_graph.clear_output()
    # Computes the number of subplots
    parameters = list(subplots.keys())
    n_parameters = len(parameters)
    n_lines = n_parameters//2
    if n_parameters%2 == 0:
        None
    else:
        n_lines += 1
    with out_graph:
        fig, axs = plt.subplots(n_lines, 2, figsize=(8 ,n_lines*4))
        for index, parameter in enumerate(parameters):
            plt.subplot(n_lines, 2, index+1)
            list_data = subplots[parameter]
            for data in list_data:
                x, y, label = data
                if label.endswith('.txt') or label.endswith('.csv'):
                    label = label[:-4] # Deletes the extension
                if len(x)>15:
                    # Continuous representation of results
                    plt.plot(x, y, label=label)
                else:
                    # Discrete representation of results
                    plt.plot(x, y, '.', label=label)
            plt.xlabel("volume fraction")
            plt.ylabel(parameter)
            plt.title("Model comparison - "+parameter)
            if parameter=='nu':
                plt.title("Model comparison - $\\nu$")
            plt.xlim(0,1)
            plt.legend()
        plt.show()


out_graph = widgets.Output()
subplot_data = {} # Format: {parameter: subplot_data}, with subplot_data = [[f_list, values, label]] and as many lists as models to plot
plotted = [] # Already added models and text files
fig = None

# Selection of the volume fraction to play with
display(widgets.Label(value="Select an inclusion, then click on 'Start comparing'. The results of the compared models will be plotted against the volume fraction of this inclusion."))
w_inclusion = widgets.Dropdown(layout={'width': 'max-content'})
w_setgraph = widgets.Button(description='Start comparing / Reset graph', layout={'width': 'max-content'})
display(widgets.HBox([w_inclusion, w_setgraph]))

def update_inclusions_list(b):
    """
    Called when a microstructure is generated.
    Updates the list of the microstructure inclusions.
    """
    global subplot_data, plotted
    inclusions = microstructure.dict_inclusions
    # Inclusions and interpahses separation
    options = {}
    for instance in inclusions:
        if type(instance)==Inclusion:
            options[instance.name] = instance
        else:
            options[instance.inclusion.name] = instance.inclusion
            options[instance.interphase.name] = instance.interphase
    w_inclusion.options = options
    out_graph.clear_output()
    subplot_data = {}
    plotted = []

if microstructure!= None:
    # Avoids the error encountered when the microstructure doesn't exist yet
    update_inclusions_list(None)
b_generate_structure.on_click(update_inclusions_list)

# Computation of a consistent list of volume fractions 'f_list'
f_list = [] 
inclusion = None # Selected inclusion

def compute_f_list(b):
    """
    Called when the 'Start comparing' button is toggled.
    Recovers the selected inclusion/interphase and computes a volume fraction list 'f_list' compatible with the rest of the inclusions.
    """
    global f_list, inclusion, subplot_data, plotted
    inclusion = w_inclusion.value
    # Computation of f_max
    f_max = 0.99
    npoints = 51
    dict_inclusions = microstructure.dict_inclusions
    for other_inclusion, f in list(dict_inclusions.items()):
        # Simple inclusions
        if type(other_inclusion)==Inclusion and other_inclusion!=inclusion:
            f_max -= f
        # Inclusions with interphase
        elif type(other_inclusion)==InclusionAndInterphase:
            if other_inclusion.inclusion!=inclusion:
                f_max -= f[0]
            if other_inclusion.interphase!=inclusion:
                f_max -= f[1]
    f_list = np.linspace(0.01, f_max, npoints)
    out_graph.clear_output()
    subplot_data = {}
    plotted = []

w_setgraph.on_click(compute_f_list)

# Adding models to graph
display(widgets.Label(value="Select a model to plot and click the 'Add model' button."))
w_addmodel = widgets.Button(description="Add model")
display(widgets.HBox([select_model, w_addmodel]))

def plot_model(b):
    """
    Called when a model is added.
    Checks whether the model has not been added yet, adds it if it hasn't been.
    Updates the plot.
    """
    global subplot_data, plotted, f_list, inclusion, fig
    model = select_model.value
    if inclusion == None:
        return None
    if model not in plotted:
        ### Saving the initial volume fraction value
        # Recovery of the associated instance when the inclusion is part of an InterphaseAndInclusion instance
        if inclusion.inc_and_int==None:
            effective_inclusion = inclusion
        else:
            effective_inclusion = inclusion.inc_and_int[0]
        f_old = microstructure.dict_inclusions[effective_inclusion] 
        plotted.append(model)
        # Computation of the list of the behaviors for the different values of f
        list_behaviors = {} # Format: {parameter: [values against f]}
        for f in f_list:
            # Simple inclusion
            if inclusion.inc_and_int == None:
                microstructure.change_fi(inclusion, f)
            # Inclusion with an interphase
            else:
                instance, pos = inclusion.inc_and_int # pos equals to 0 for an inclusion and 1 for the interphase
                # Computes the new volume fraction list for the InclusionAndInterphase instance
                new_f = microstructure.dict_inclusions[instance]
                new_f[pos] = f
                # Incrementation of the volume fraction
                microstructure.change_fi(instance, new_f)
            h_behavior = model.compute_h_behavior(microstructure)
            if 'K' not in list(h_behavior.keys()):
                h_behavior = Isotropic_behavior(h_behavior)
                
            for parameter, value in h_behavior.items():
                if parameter not in list_behaviors.keys():
                    # The parameter is encountered for the first time
                    list_behaviors[parameter] = []
                list_behaviors[parameter].append(value)
        # Update of the 'subplot_data' database
        for parameter, values in list_behaviors.items():
            data = [f_list, values, model.name]
            # Creation of the parameter entry when it is ploted for the first time
            if parameter not in subplot_data.keys():
                subplot_data[parameter] = []
            subplot_data[parameter].append(data)
        # Reversing the microstructure changes
        microstructure.change_fi(effective_inclusion, f_old)
        # Plotting data
        draw_all_data(subplot_data)
                    
w_addmodel.on_click(plot_model)

# Adding data from text files
display(widgets.Label(value="Plot data from a text file. Input files are in the 'inputs/model_comparison' folder. See 'example.txt' for the format."))
list_files = listdir('inputs/model_comparison/')
w_file = widgets.Dropdown(options=[file for file in list_files if file.endswith('.txt')])
w_add_data = widgets.Button(description="Add data")
display(widgets.HBox([w_file, w_add_data]))

def plot_data(b):
    """
    Called when the 'Add data' button is toggled.
    Checks wheteher the selected file has been added to the graph, adds it if it hasn't been.
    Updated the graph.
    """
    global subplot_data, plotted, fig
    file_name = w_file.value
    if file_name not in plotted:
        plotted.append(file_name)
        df = pd.read_csv('inputs/model_comparison/'+file_name)
        # Recovery of the volume fraction list
        try:
            f_values = df['f']
        except KeyError:
            with out_graph:
                print("Wrong format")
        # Parameters recovery
        parameters = df.keys()
        for parameter in parameters:
            if parameter == 'f':
                continue
            # Creation of the parameter entry
            if parameter not in subplot_data.keys():
                subplot_data[parameter] = []
            values = list(df[parameter])
            subplot_data[parameter].append([f_values, values, file_name]) 
        # Graph plotting
        draw_all_data(subplot_data)
    
w_add_data.on_click(plot_data)

# Hashin bounds
w_addbounds = widgets.Button(description="Add HS bounds")
display(widgets.Label(value="Add Hashin-Shtrikman bounds to figures"))
display(w_addbounds)

def plot_bounds(b):
    """
    Called when the ashin bounds are added.
    Adds the Hashin bounds to the graph if they have not been added yet.
    Updates the graph.
    """
    global subplot_data, plotted, f_list, inclusion
    if "bounds" not in plotted and microstructure.Hashin_bounds()!=None:
        plotted.append("bounds")
        f_old = microstructure.dict_inclusions[inclusion] # Savec the initial volume fraction value
        # Computes the list of behaviors for the different values of f
        list_behaviors = {} # Format: {parameter: [values against f]}
        for f in f_list:
            microstructure.change_fi(inclusion, f)
            h_bounds = microstructure.Hashin_bounds()
            for parameter, value in h_bounds.items():
                if parameter not in list_behaviors.keys():
                    # First encounter of the parameter
                    list_behaviors[parameter] = []
                list_behaviors[parameter].append(value)
        # Update of the 'subplot_data' database
        for parameter, values in list_behaviors.items():
            parameter_key = parameter[:-3] # Deletes the 'inf' and 'sup' strings
            parameter_suffix = parameter[-3:] # 'inf' or 'sup'
            if parameter_suffix=='inf':
                suffix = 'lower'
            else:
                suffix = 'upper'
            label = "HS " + suffix + " bound " + parameter_key
            data = [f_list, values, label]
            # Creates the entry parameter if it is plotted for the first times
            if parameter_key not in subplot_data.keys():
                subplot_data[parameter_key] = []
            subplot_data[parameter_key].append(data)
        # Reverse the microstructure changes
        microstructure.change_fi(inclusion, f_old)
        # Drawing the data
        draw_all_data(subplot_data)

w_addbounds.on_click(plot_bounds)

# Saving the figures
n_fig = 0 # Figure index
w_save_image = widgets.Button(description="Save figures")
w_filename = widgets.Text(value='fig0.pdf')
display(widgets.Label(value="Enter a valid file name with an extension (ex: .pdf, .png, .jpg) and click the save button to save the figures. The output file will be saved in the 'outputs/model_comparison' folder"))
display(widgets.HBox([w_filename, w_save_image]))

def save_image(b):
    """
    Called when a figure is saved. Saves the figure in the 'outputs/model_comparison' fodler with the selected name.
    Automatically changes the name of the next figure to be saved.
    """
    global fig, n_fig
    filename = w_filename.value
    fig.savefig("outputs/model_comparison/"+filename)
    # Automatic update of the name
    n_fig += 1
    w_filename.value = 'fig{}.pdf'.format(n_fig)
    
w_save_image.on_click(save_image)

# Saving the data
n_data = 0 # Index of the data file
w_save_data = widgets.Button(description="Save data")
w_data_filename = widgets.Text(value='data0.csv')
display(widgets.Label(value="Enter a valid file name with an extension (ex: .txt, .csv) and click the save button to save the figures data. The ouput file will be saved in the 'outputs/model_comparison' folder"))
display(widgets.HBox([w_data_filename, w_save_data]))

def save_data(b):
    """
    Called when data is saved. Saves the data (txt) in the 'ouputs.model_comparison' folder with the selected name.
    Automatically changes the name of the next data file to be saved.
    """ 
    global subplot_data, n_data
    filename = w_data_filename.value
    # Automatic update of the name
    n_data += 1
    w_data_filename.value = 'data{}.csv'.format(n_data)
    # Creation of a dataframe containing the data
    data = {}
    # Recovery of the models data 
    for parameter, parameter_data in subplot_data.items():
        for model_data in parameter_data:
            f_list, model_values, label = model_data
            if label.endswith('.txt') == False and label.endswith('.csv') == False:
                # The data refers to a model and not to an input file
                data["volume fraction f"] = f_list
                data[parameter + " - " + label] = model_values
    df = pd.DataFrame(data)
    df.to_csv('outputs/model_comparison/'+filename, header=True, index=False, sep=',', mode='a')
    
w_save_data.on_click(save_data)

display(out_graph)

Label(value="Select an inclusion, then click on 'Start comparing'. The results of the compared models will be …

Label(value="Select a model to plot and click the 'Add model' button.")

Label(value="Plot data from a text file. Input files are in the 'inputs/model_comparison' folder. See 'example…

Label(value='Add Hashin-Shtrikman bounds to figures')

Button(description='Add HS bounds', style=ButtonStyle())

Label(value="Enter a valid file name with an extension (ex: .pdf, .png, .jpg) and click the save button to sav…

Label(value="Enter a valid file name with an extension (ex: .txt, .csv) and click the save button to save the …

Output()

---

## IV- Automatic computation from a .txt file
TODO : Description de la section

TODO : Décrire les fichiers compatibles et le format voulu, demander à l'utilisateur de mettre ses fichiers dans le dossier inputs 

TODO : Réfléchir à un format pertinent des fichiers d'entrée 

TODO : Ajout d'une barre de progression (utile pour les longs fichiers)

In [10]:
list_inputs = [] # List of the compatible files in the input folder
folder = 'inputs/automatic_computation/' # Input folder

# Recovery of the files
def compatible_file(file_name, folder):
    """
    Checks whether a given file 'file_name' in the 'folder' folder has the wanted format, meaning:
    - the file is a .txt file
    - its first line is '*homogenisation'
    Returns a bool.
    """
    result = True # Initialisation
    # File name test
    if len(file_name)<5 or file_name[-4:]!='.txt':
        result = False
    else:
        # First line reading
        with open(folder+file_name, 'r') as file:
            line = file.readline()
            if line.strip() != '*homogenisation':
                result = False
    return result

def check_files(folder=folder):
    """
    Called when the 'refresh list' button is toggled.
    Updates the 'list_inputs' list of compatible files in the input folder.
    """
    global list_inputs
    list_inputs_raw = listdir(folder)
    list_inputs = [] # Resets the list
    for file_name in list_inputs_raw:
        if compatible_file(file_name, folder):
            list_inputs.append(file_name)

def read_file(b):
    """
    Called when the 'Generate output file' button is toggled.
    Reads the name of the selected file, opens and reads the file.
    Displays an error message if an error is detected.
    Else, computes the homogenised behavior of each microstructure and creates an output file in the 'outputs/automatic_computation' folder.
    TODO: adapter au calcul avec des modèles différents donnant des paramètres de comportement différents
    """
    folder_in = 'inputs/automatic_computation/'
    folder_out = 'outputs/automatic_computation/'
    file_name = w_file.value
    file_name_out = file_name[:-4] + '_out.csv' # Adds the '.csv' extension
    out_file.clear_output()
    read_matrix = False # Defines whether the current line refers to the definition of a new microstructure or of an inclusion
    read_model = True # Defines if the active line refers to a model selection
    dict_inclusions = {}
    n = 0
    # Output file initialisation
    with open(folder_out+file_name_out, 'w') as file_out:
        file_out.write("K,G,E,nu\n")
    # Reading the input file
    with open(folder_in+file_name, 'r') as file:
        lines = file.readlines()
        for n_line, line in enumerate(lines[1:]):
            try:
                if read_model:
                    # Model definition
                    model_name = line.strip()
                    model = str_to_model(model_name)
                    # Going to next line
                    read_model = False
                    read_matrix = True
                elif read_matrix:
                    # Matrix behavior reading
                    matrix_behavior = {}
                    line1 = line.strip().split(',')
                    for parameter in line1:
                        parameter = parameter.split(':')
                        matrix_behavior[parameter[0]] = float(parameter[1])
                    # Going to next line
                    read_matrix = False
                elif line.strip() == '*':
                    # Computation of the previous microstructure's homogenised behavior
                    microstructure = Microstructure(matrix_behavior, dict_inclusions)
                    behavior_h = model.compute_h_behavior(microstructure)
                    # Writing the behavior in the output file
                    with open(folder_out+file_name_out, 'a') as file_out:
                        values = list(behavior_h.values())
                        values = [str(e) for e in values]
                        file_out.write(','.join(values)+'\n')
                    # Going to next line
                    read_model = True
                    dict_inclusions = {}
                    n += 1
                else:
                    # Reading an inclusion
                    line1 = line.strip().split(',')
                    type_inclusion = line1[0]
                    f = line1[-1] # volume fraction
                    inclusion_behavior = {}
                    for parameter in line1[1:-1]:
                        parameter = parameter.strip().split(':')
                        inclusion_behavior[parameter[0]] = float(parameter[1])
                    # Inclusion generation
                    inclusion = Inclusion(int(type_inclusion), inclusion_behavior)
                    dict_inclusions[inclusion] = float(f)
            except:
                with out_file:
                    print("Error on line {}: {} ".format(n_line+1, line))
                    return None
    with out_file:
        print("Output file generated in the 'outputs/automatic_computation' folder ")

def refresh(b):
    """
    Called when the 'Refresh input files' button is toggled.
    Updates the compatible input files list and widget.
    """
    check_files()
    w_file.options = list_inputs

b_refresh = widgets.Button(description='Refresh input files list')
display(b_refresh)
w_label = widgets.Label(value='Choose an input file :')
w_file = widgets.Dropdown(options=list_inputs)
refresh(None) # Update of the available input files
b_compute = widgets.Button(description='Generate output file')
display(widgets.HBox([w_label, w_file, b_compute]))
out_file = widgets.Output(layout={'width': 'max-content', 'border': '1px solid #FF625BF5'})
display(out_file)
out_file.clear_output()
with out_file:
    print("Press 'Generate output file' to compute ")
    
b_refresh.on_click(refresh)
b_compute.on_click(read_file)

message = """If your file does not appear :
- Check that it is a '.txt' file and that its first line is '*homogeneisation',
- Check that your file is in the 'inputs' folder,
- Press the 'Refresh input files list' button."""

print(message)


Button(description='Refresh input files list', style=ButtonStyle())

Output(layout=Layout(border='1px solid #FF625BF5', width='max-content'))

If your file does not appear :
- Check that it is a '.txt' file and that its first line is '*homogeneisation',
- Check that your file is in the 'inputs' folder,
- Press the 'Refresh input files list' button.


---

## V- Model description

To add a model here, simply write its description in a Markdown file (.md) in the 'model_descriptions' folder.

The first line must be:

'# Model name'


In [ ]:
from IPython.display import Latex, Markdown
from os import listdir

# Recovery of the model descriptions files  
folder = 'model_descriptions/'
folder_files = listdir(folder)
descriptions = [] # List of model descriptions files, format: ['model_name', 'file_path']
for file_name in folder_files:
    if file_name.endswith('.md'):
        path = folder + file_name
        with open(path, 'r') as opened_file:
            title = opened_file.readline()
        model_name = title[2:].strip() # Deletes the '#' at the beginning
        descriptions.append((model_name, path))

# Displays the description
w_description = widgets.Dropdown(options=descriptions)
display(w_description)
out_description = widgets.Output(layout=layout)
display(out_description)

def display_description(change):
    """
    Called when a model is selected.
    Recovers the model and displays its description.
    """
    out_description.clear_output()
    file_name = w_description.value
    with open(file_name, 'r') as file:
        description = file.read()
        with out_description:
            display(Markdown(description))
            
display_description(None)
w_description.observe(display_description, names='value')


## VI- Estimates of parameters by an inverse method

This section enables the computation of the ideal values of parameters such as to reach a target behavior set by the user. The user first enters a target homogenised behavior he would like to achieve. He then builds a microstructure by choosing the inclusions, their type, their behavior and volume fractions, along with the behavior of the matrix. At each step, the user can set a parameter as unknown, meaning he wants the parameter to be optimized.


In [ ]:
def gen_tab_behavior(inverse=False):
    """
    Generate a 'tab' widget defining the behavior parameters.
    Each generated tab refers to a behavior from the 'dict_behaviors' dictionnary defined in 'classes.py'.
    The 'isotropic' parameter is 'True' if one only wants to display isotropic behaviors.
    The function returns:
    - A list of list of widgets 'list_widgets' with as many lists as tabs. Each list contains the widgets for the parameters associated with the behavior of the corresponding tab.
    - The tab widget to be displayed.
    """
    behaviors_str = list(dict_behaviors.keys()) # Lists the names of the behaviors implemented in classes.py
    list_widgets = [] # List of lists, each list refers to a tab of the final widget and contains the non-formatted widgets of this tab
    tab_titles = [] # Tabs names, each tab refers to a behavior type ('Isotropic K and G', 'Anisotropic', etc..) 
    # Generation of the widgets for each tab
    for behavior_str in behaviors_str:
        if 'Anisotropic' in behavior_str:
            continue
        widgets_onglet = []
        parameters = dict_behaviors[behavior_str] # Behavior-associated parameters (example : ['K', 'G'] for Isotropic)
        for parameter in parameters:
            w = widgets.BoundedFloatText(value=200, min=0, max=10**12, step=1) # 'parameter' associated widget
            if parameter == 'nu':
                w.max = 0.5
                w.value = 0.3
                w.step = 0.1
            w_label = widgets.Label(value=parameters_name[parameter])
            if parameter != 'nu' and inverse:
                w_checkbox = widgets.Checkbox(value=False, description="Set as unknown")
                widget_onglet = widgets.HBox([w_label, w, w_checkbox])
            else:
                widget_onglet = widgets.HBox([w_label, w])
            widgets_onglet.append(widget_onglet)
        list_widgets.append(widgets_onglet)
        tab_titles.append(behavior_str)
    # Tab creation
    tab = widgets.Tab()
    tab.children = [widgets.VBox(w) for w in list_widgets]
    for pos, title in enumerate(tab_titles):
        tab.set_title(pos, title)
    return list_widgets, tab

def read_behavior(tab, list_widgets, inverse=False):
    """
    Reads tab widgets created by the 'gen_tab_behavior'.
    Returns a 'behavior' dictionnary and a list of unknown parameters if 'inverse' is True.
    """
    behavior_int = tab.selected_index # Index of the tab selected by the user
    widgets_parameters = list_widgets[behavior_int] # Parameter widgets of the active tab
    behavior = {}
    unknown_parameters = [] # Unknown parameters list
    for w in widgets_parameters:
        name = w.children[0].value
        value = w.children[1].value
        # Parameter true name
        name = parameters_name_bis[name]
        # Entry creation
        if inverse:
            try:
                # Avoids the error due to the absence of the 'set as unknown' checkbox (nu)
                unknown = w.children[2].value
                if unknown:
                    # The parameter is unknown
                    unknown_parameters.append(name)
            except:
                None
        behavior[name] = value
    if inverse:
        return behavior, unknown_parameters
    else:
        return behavior

### Target behavior
w_target = [] # Target behavior widgets list
target_behavior = {} # Initialisation
ddl = {} # List of the degrees of freedom of the optimization algorithm
# Format: {function: [min, max]}
# function takes three input arguments, an increment (float), a min and a max
# It increments a parameter (ex: volume fraction of an inclusion) if it stays between its bounds
widgets_target, tab_target = gen_tab_behavior()
b_target = widgets.Button(description="Generate target behavior", layout={'width': 'max-content'})
out_target = widgets.Output()


# Behavior generation
def generate_target(b):
    """
    Called when a target behavior is generated.
    """
    global target_behavior
    # Recovery of the selected behavior
    out_target.clear_output()
    target_behavior = read_behavior(tab_target, widgets_target)
    with out_target:
        print("Target behavior generated : {}".format(target_behavior))

b_target.on_click(generate_target)

# Displays the target widgets
w_target = [tab_target, b_target, out_target]
display(Markdown("## Target behavior"))
display(widgets.VBox(w_target, layout=layout))

### Inclusion generation
dict_inclusions = {}
# Initialisation of the list of created inclusions. Format: {name_inclusion (str): inclusion (Inclusion or InclusionAndInterphase)}
unknown_inclusion_parameters = {}
# Contains the unknown parameters of each inclusion, format: {inclusion: [unknown parameters]}

# Inclusion name
w_label = widgets.Label(value='Inclusion name')
n_inclusion = 0 # Index used to give each inclusion a unique name
w_name = widgets.Text(value='inclusion'+str(n_inclusion))

# Inclusion type
caption_type = widgets.Label(value='Inclusion type')
widgets_type, tab_type = gen_tab_type() # Inclusion type tab generation

# Inclusion behavior
caption = widgets.Label(value='Inclusion behavior')
list_widgets, tab = gen_tab_behavior(inverse=True)

# Inclusion generation
button_generate_inclusion = widgets.Button(description="Generate Inclusion")
output = widgets.Output()
def generate_inclusion(b):
    """
    Called when an inclusion is generated, creates an inclusion with the selected parameters.
    """
    global n_inclusion, unknown_inclusion_parameters
    # Recovery of the selected inclusions
    output.clear_output()
    inclusion_name = w_name.value
    if inclusion_name in list(dict_inclusions.keys()):
        with output:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type, widgets_type)
        behavior, unknown_parameters = read_behavior(tab, list_widgets, inverse=True)
        inclusion = Inclusion(type_inclusion, behavior, name=inclusion_name, aspect_ratio=inclusion_aspect_ratio)
        dict_inclusions[inclusion_name] = inclusion
        w_inclusions_info.options = list(dict_inclusions.keys())
        with output:
            print("Inclusion generated: ", inclusion)
            print("Unknown parameters: ", unknown_parameters)
        # Automatic update of the inclusion name
        n_inclusion += 1
        w_name.value = 'inclusion'+str(n_inclusion)
        # Saves the unknown parameters
        unknown_inclusion_parameters[inclusion] = unknown_parameters
    
button_generate_inclusion.on_click(generate_inclusion)

# Displays the inclusion widgets 
w_inclusions = [w_label, w_name, caption_type, tab_type, caption, tab, button_generate_inclusion, output]
display(Markdown("## Inclusion generation"))
display(widgets.VBox(w_inclusions, layout=layout))

### Inclusion and interpahse generation
# Inclusion name
w_label_bis = widgets.Label(value='Instance name')
n_inclusion_bis = 0 # Index used to give each instance a unique name
w_name_bis = widgets.Text(value='microstructure'+str(n_inclusion_bis))

# Inclusion type
widgets_type_bis, tab_type_bis = gen_tab_type() # InclusionAndInterpahse type tab generation

# Inclusion behavior
caption_incl = widgets.Label(value='Inclusion behavior')
list_widgets_incl, tab_incl = gen_tab_behavior(inverse=True)
# Interphase behavior
caption_inter = widgets.Label(value='Interphase behavior')
list_widgets_inter, tab_inter = gen_tab_behavior(inverse=True)

# Inclusion generation
button_generate_inclusion_bis = widgets.Button(description="Generate inclusion with interphase", layout={'width': 'max-content'})
output_bis = widgets.Output()

def generate_inclusion_bis(b):
    """
    Called when an inclusion with interphase is generated. Creates an InclusionAndInterphase instance.
    """
    global n_inclusion_bis
    # Recovery of the selected parameters
    output_bis.clear_output()
    instance_name = w_name_bis.value
    if instance_name in list(dict_inclusions.keys()):
        with output_bis:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type_bis, widgets_type_bis)
        behavior_incl, unknown_parameters_incl = read_behavior(tab_incl, list_widgets_incl, inverse=True) # behavior of the inclusion
        behavior_inter, unknown_parameters_inter = read_behavior(tab_inter, list_widgets_inter, inverse=True) # behavior of the interphase
        inclusion = Inclusion(type_inclusion, behavior_incl, name=instance_name+'_inclusion', aspect_ratio=inclusion_aspect_ratio) # Creation of the inclusion
        interphase = Inclusion(type_inclusion, behavior_inter, name=instance_name+'_interphase', aspect_ratio=inclusion_aspect_ratio) # Creation of the interphase
        instance = InclusionAndInterphase(inclusion, interphase, name=instance_name) # Creation of the instance InclusionAndInterphase
        # Inclusions list update
        dict_inclusions[instance_name] = instance
        w_inclusions_info.options = list(dict_inclusions.keys())
        # Unknwon parameters dict update
        unknown_inclusion_parameters[inclusion] = unknown_parameters_incl
        unknown_inclusion_parameters[interphase] = unknown_parameters_inter
        with output_bis:
            print("Inclusion generated: ", instance)
            print("Unknown parameters - inclusion: ", unknown_parameters_incl)
            print("Unknown parameters - interphase: ", unknown_parameters_inter)
        # Automatic update of the instance name
        n_inclusion_bis += 1
        w_name_bis.value = 'microstructure'+str(n_inclusion_bis)
    
button_generate_inclusion_bis.on_click(generate_inclusion_bis)
display(Markdown("## Inclusion and interphase generation"))
w_inclusion_bis = widgets.VBox([w_label_bis,
                                w_name_bis,
                                widgets.Label(value='Inclusion type'),
                                tab_type_bis,
                                caption_incl,
                                tab_incl,
                                caption_inter,
                                tab_inter,
                                button_generate_inclusion_bis,
                                output_bis],
                          layout=layout)
display(w_inclusion_bis)

### Inclusions info and deletion
w_label = widgets.Label(value="Displays info on the generated inclusions")
w_inclusions_info = widgets.Dropdown(options=list(dict_inclusions.keys()), layout={'width': 'max-content'})
w_delete = widgets.Button(description="Delete inclusion", layout={'width': 'max-content'})
out_inclusions_info = widgets.Output()
# Displays the selected inclusion info
def display_info(change):
    """
    Called when an inclusion is selected, displays its description.
    """
    out_inclusions_info.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions_info.value]
    except KeyError:
        inclusion = None
    with out_inclusions_info:
        print(inclusion)
        
w_inclusions_info.observe(display_info, names='value')
# Inclusion deletion
def delete_inclusion(b):
    """
    Called when an inclusion is deleted.
    Recovers the inclusion and deletes it from the generated inclusions dict.
    """
    inclusion_name = w_inclusions_info.value
    try:
        del dict_inclusions[inclusion_name]
    except KeyError:
        None
    w_inclusions.options = list(dict_inclusions.keys())
    w_inclusions_info.options = list(dict_inclusions.keys())

w_delete.on_click(delete_inclusion)
# Displays inclusions info widgets
display(Markdown("## Inclusions info"))
display(widgets.VBox([w_label, widgets.HBox([w_inclusions_info, w_delete]), out_inclusions_info], layout=layout))

### Microstructure generation
microstructure = None # Initialisation

def add_inclusion_to_structure(b):
    """
    Called when the 'Add inclusion' button is toggled.
    Creates a widget linked to the volume fraction of the selected inclusion and adds it to the 'widgets_f' dict.
    Also creates a 'Remove inclusion' button and adds it to the 'buttons' dict.
    Eventually displays the generated widgets.
    """
    out2.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions.value]
    except KeyError:
        return None
    if inclusion in list(widgets_f.keys()):
        with out2:
            print("Already added")
    else:
        w_name = widgets.Label(inclusion.name) # Added inclusion name
        w_b = widgets.Button(description="Remove inclusion")
        w_b.on_click(remove_inclusion)
        buttons_suppress[w_b] = inclusion # Button and inclusion association
        # Simple inclusions
        if type(inclusion)==Inclusion:
            w_f = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f') # volume fraction widget
            w_c = widgets.Checkbox(value=False, description="Set as unknown")
            widgets_f[inclusion] = (w_name, w_f, w_c) # Widgets referring to the inclusion
            with out1:
                display(widgets.HBox([w_name, w_b]), widgets.HBox([w_f, w_c]))
        # Inclusions with interphase
        else:
            w_f_incl = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f_inclusion')
            w_f_int = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f_interphase')
            w_c_incl = widgets.Checkbox(value=False, description="Set as unknown")
            w_c_int = widgets.Checkbox(value=False, description="Set as unknown")
            widgets_f[inclusion] = (w_name, w_f_incl, w_c_incl, w_f_int, w_c_int) # Widgets referring to the inclusion
            with out1:
                display(widgets.HBox([w_name, w_b]), widgets.HBox([w_f_incl, w_c_incl]), widgets.HBox([w_f_int, w_c_int]))
            
def add_inclusion_to_list(b):
    """
    Called when an inclusion or inclusion and interphase is generated.
    Updates the inclusion selection widgets.
    """
    w_inclusions.options = list(dict_inclusions.keys())
    
def remove_inclusion(b):
    """
    Called when an inclusion is removed from the structure.
    Recovers the selected inclusion, closes its widgets and removes it from the 'widgets_f' dict.
    """
    out2.clear_output()
    inclusion = buttons_suppress[b] # Recovery of the inclusion associated to the inclusion
    # Closing widgets
    for widget in widgets_f[inclusion]:
        widget.close()
    b.close()
    # Widgets list update
    del widgets_f[inclusion]
    del buttons_suppress[b]

# Dynamic generation of functions associated to degrees of freedom
def make_function_f(inclusion, index=None):
    """
    Creates a function that change an inclusion volume fraction
    """
    def function(value_incr, mini, maxi):
        """
        Increments the value of the unknown volume fraction by 'value_incr' (float)
        mini and maxi are the bounds of the parameter.
        Returns a True boolean if the modified value of the parameter is within the bounds.
        """
        if index==None:
            value = microstructure.dict_inclusions[inclusion]
            value, changed = incr(value, value_incr, mini, maxi)
            microstructure.change_fi(inclusion, value)
        else:
            value = microstructure.dict_inclusions[inclusion]
            value[index], changed = incr(value[index], value_incr, mini, maxi)
            microstructure.change_fi(inclusion, value)
        return changed
    return function

def make_function_mbehavior(parameter):
    """
    Creates a function that modifies the value of a matrix behavior parameter.
    parameter: str (example : 'K', 'G')
    """
    def function(value_incr, mini, maxi):
        value = microstructure.behavior[parameter]
        value, changed = incr(value, value_incr, mini, maxi)
        microstructure.change_parameter(parameter, value)
        return changed
    return function

def make_function_ibehavior(inclusion, parameter):
    """
    Creates a function that modifies the value of the inclusion parameter.
    parameter: str (example : 'K', 'G')
    """ 
    def function(value_incr, mini, maxi):
        value = inclusion.behavior[parameter]
        value, changed = incr(value, value_incr, mini, maxi)
        inclusion.change_parameter(parameter, value)
        return changed
    return function

def generate_microstructure(b):
    """
    Generates a microstructure with the parameters set by the user.
    Displays an error message if the chosen volume fractions are not consistent.
    Eventually displays a description of the generated microstructure.
    Updates the degrees of freedom list for the optimization algorithm.
    Met à jour la liste des degrés de liberté de l'algorithme d'optimization.
    """
    global microstructure, ddl
    ddl = {}
    matrix_behavior, unknown_parameters = read_behavior(tab_m, widgets_m, inverse=True) # Reading the 'Matrix beahvior' widgets
    dict_inclusions = {}
    # Generation of the unknown parameters functions
    for parameter in unknown_parameters:
        function = make_function_mbehavior(parameter)
        ddl[function] = [0.01, 10**6, 0.05]
    # Adding inclusions
    dict_inclusions = {}
    # Reading the selected volume fractions
    for inclusion, widgets in widgets_f.items():
        # Simple inclusions
        if type(inclusion)==Inclusion:
            w_name, w_f, w_c = widgets
            f = w_f.value # Selected volume fraction
            unknown_f = w_c.value # True if f is unknwon
            if unknown_f:
                f = 0.01 # Initialisation of the unknown value
                # Definition of the function associated to the degree of freedom
                function = make_function_f(inclusion)
                ddl[function] = [0.01, 0.99, 0.0001]
            dict_inclusions[inclusion] = f # Generation of the microstructure attribute
        # Inclusions with interphase
        else:
            w_name, w_f_inc, w_c_inc, w_f_int, w_c_int = widgets
            f_inc, f_int = w_f_inc.value, w_f_int.value
            unknown_f_inc, unknown_f_int = w_c_inc.value, w_c_int.value
            if unknown_f_inc:
                f_inc = 0.01 # Initialisation of the unknown parameter
                # Definition of the function associated to the degree of freedom
                function = make_function_f(inclusion, index=0)
                ddl[function] = [0.01, 0.99, 0.0001]
            if unknown_f_int:
                f_int = 0.01 # Initialisation of the unknown value
                # Definition of the function associated to the degree of freedom
                function = make_function_f(inclusion, index=1)
                ddl[function] = [0.01, 0.99, 0.0001]
            dict_inclusions[inclusion] = [f_inc, f_int] # Generation of the microstructure attribute
    # Generation of functions associated to unknown inclusion behavior parameters
    instances = [] # Instance list (inclusions or interphases)
    for inclusion in dict_inclusions.keys():
        if type(inclusion)==Inclusion:
            instances.append(inclusion)
        else:
            instances += [inclusion.inclusion, inclusion.interphase]
    for inclusion in instances:
        try:
            unknown_parameters = unknown_inclusion_parameters[inclusion]
            for parameter in unknown_parameters:
                function = make_function_ibehavior(inclusion, parameter)
                ddl[function] = [0.01, 10**6, 0.05]
        except KeyError:
            # All the inclusion parameters are known
            None
    # Microstructure generation
    out3.clear_output()
    try:
        microstructure = Microstructure(matrix_behavior, dict_inclusions)
        with out3:
            print("Microstructure generated\n" + str(microstructure))
            #print("Unknown parameters: ", unknown_parameters)
    except NameError:
        microstructure = None
        with out3:
            print("Inconsistent choice of volume fractions")

# Matrix behavior
caption = widgets.Label(value='Matrix behavior')
widgets_m, tab_m = gen_tab_behavior(inverse=True)

# Adding inclusions
w_inclusions = widgets.Dropdown(options=list(dict_inclusions.values()), layout={'width': 'max-content'})
button_add_inclusion = widgets.Button(description="Add inclusion")
out1 = widgets.Output() # Displays volume fractions widgets
out2 = widgets.Output() # Displays 'already added inclusion' messages
widgets_f = {} # Contains already added inclusions and their widgets ('name','volume fraction')
buttons_suppress = {} # 'Remove inclusion' buttons and their associated inclusions

button_add_inclusion.on_click(add_inclusion_to_structure)
button_generate_inclusion.on_click(add_inclusion_to_list)
button_generate_inclusion_bis.on_click(add_inclusion_to_list)

# Microstructure generation
b_generate_structure = widgets.Button(description='Generate microstructure', layout={'width': 'max-content'})
# TODO : widget 'valid' qui indique en temps réel si les fractions volumiques choisies sont cohérentes
out3 = widgets.Output()
b_generate_structure.on_click(generate_microstructure)

# Displays microstructure widgets
w_micro = [caption, tab_m, widgets.HBox([w_inclusions, button_add_inclusion, out2]), out1, widgets.HBox([b_generate_structure]),out3]
display(Markdown("## Microstructure generation"))
display(widgets.VBox(w_micro, layout=layout))

def compute_error(microstructure, model, target):
    """
    Compute the least squares error between the target and the homogenised beahviors.
    """
    behavior_h = model.compute_h_behavior(microstructure)
    difference = [target[parameter]-behavior_h[parameter] for parameter in target.keys()]
    return np.linalg.norm(difference)

def grad(target, model, ddl, error_old):
    """
    target: dict, target behavior
    model: Model instance
    microstructure: Microstructure instance
    ddl: dict, contains the optimization problem unknown parameters and their bounds
    error_old: float, error computed for the on-going iteration
    """
    global microstructure
    result = {} # format: {variable function: value of the gradient}
    # Term by term gradient computation
    for function, min_max in ddl.items():
        mini, maxi, pas = min_max # Variable bounds
        pas_i = (maxi - mini)*pas # Common to all the parameters
        changed = function(pas_i, mini, maxi) # Parameter update
        # New error computation
        error_new = compute_error(microstructure, model, target)
        # Parameter gradient
        result[function] = (error_new - error_old)/pas_i
        if changed:
            function(-pas_i, mini, maxi)
    return result

def optimise(target, model, ddl):
    """
    Uses the gradient algorithm to optimise the 'ddl' parameters and reach the target behavior.
    """
    global microstructure
    try:
        pas = list(ddl.values())[0][2] # First variable step, used for all the variables
    except:
        return None # No unknown parameters
    seuil = 10**(-3)
    criteria = seuil + 1 # Criteria initialisation
    max_iterations = 10000
    iteration = 0
    errors = []
    # Error value
    error = compute_error(microstructure, model, target)
    while criteria>seuil and iteration<max_iterations:
        # Gradient computation
        grad_error = grad(target, model, ddl, error)
        #print("iter ", iteration, " error ", error," fi ", microstructure.dict_inclusions[inclusion]," grad ", list(grad_error.values())[0])
        iteration += 1
        # TODO : coder une optimization de alpha
        # Variable update
        for function, minmax in ddl.items():
            mini, maxi = minmax[:2]
            pas_i = pas
            value_incr = -pas_i*grad_error[function]
            function(value_incr, mini, maxi) # Calcul des variables à l'itération suivante
#             print("iter ", iteration, " error ", error," Gm ", microstructure.behavior['G']," grad ", list(grad_error.values())[0]," value_incr ", value_incr)
#            print("iter ", iteration, " error ", error," fi ", microstructure.dict_inclusions," grad ", list(grad_error.values())[0]," value_incr ", value_incr)
#             print("iter ", iteration, " error ", error," value_incr ", value_incr)
        # Error update
        error = compute_error(microstructure, model, target)
        errors.append(error)
        if len(errors)>2:
            criteria = errors[-2] - errors[-1]
    behavior = model.compute_h_behavior(microstructure)
    return microstructure, errors, iteration, behavior

### Model selection
def test_models(b=None):
    """
    Called when a microstructure is generated.
    Tests the implemented models on the generated microstructure and creates a 'valid_models' list of compatible models.
    """
    valid_models = []
    if microstructure == None:
        # Checks whether the microstructure has been generated
        return None
    for Model in list_models:
        model = Model()
        valid = model.check_hypothesis(microstructure)
        if valid:
            # The microstructure fits the model hypothesis
            valid_models.append((model.name, model))
    # Updates the model selection widget
    select_model.options = valid_models

valid_models = [] # List of the microstructure-compatible models: [(model_name, Model)]
select_model = widgets.Dropdown()
test_models()
label = widgets.Label(value="Select a model. Only compatible models will be displayed.")
b_compute = widgets.Button(description='Optimise')
output_optimization = widgets.Output()

def optimise_microstructure(b):
    """
    Called when the 'optimise' button is toggled.
    Recovers the selected model, computes the homogenised behavior and displays it.
    """
    model = select_model.value
    output_optimization.clear_output()
    microstructure, errors, iteration, behavior = optimise(target_behavior, model, ddl)
    with output_optimization:
        print("optimized microstructure - {} model".format(model.name))
        print("optimization done with {} iterations".format(iteration))
        print("Target behavior: ", target_behavior)
        print("Actual behavior: ", behavior)
        print("optimized microstructure:")
        print(microstructure)
    # Microstructure reset
    generate_microstructure(None)
    

b_generate_structure.on_click(test_models)
b_compute.on_click(optimise_microstructure)

# Displays optimization widgets 
w_opti = [label, widgets.HBox([select_model, b_compute]), output_optimization]
display(Markdown("## optimization"))
display(widgets.VBox(w_opti, layout=layout))

---